#**Arabic Snippet Generation: Query based text summarization**

#Installs/Imports

In [ ]:
!pip install rouge
!pip install tashaphyne

     |████████████████████████████████| 251 kB 5.1 MB/s 
     |████████████████████████████████| 126 kB 33.0 MB/s 


In [ ]:
import json
import re
import nltk
from nltk.stem.isri import ISRIStemmer
from matplotlib.pyplot import text 
import math
nltk.download('punkt')
nltk.download('stopwords')
arb_stopwords = set(nltk.corpus.stopwords.words("arabic"))
from nltk.tokenize import word_tokenize
from nltk.tag.stanford import StanfordPOSTagger as POS_Tag
arabic_postagger = POS_Tag('arabic.tagger', 'stanford-postagger.jar')
arabic_postagger._SEPARATOR = '/'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/nltk/tag/stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)


#Helper Functions

In [ ]:
def clean_data(sentence):
  sentence = sentence.replace("[عدل]", " ")
  sentence = sentence.replace("[", " ")
  sentence = sentence.replace("]", " ")
  #sentence=sentence.replace("ال","")
  #sentence=sentence.replace("لإ","")
  
  sentence = re.sub("[\$|£|€|a-zA-Z|:|\ufeff|\‘|َ|ُ|ْ|َِِ|ّ|ً|ٌ]", "", sentence)
  sentence = re.sub("[\(|=|.|,|;|،\)|<|>|!|?|»|«|/|\+|\*|\(\)|\-|\[|\]|\(|\)|\{|\}|_|é|ù|è|؛|–|’\|/|؛|'\|…|ـ|&|؟|%|\“|\"|—|\”|@]"," ", sentence)
  sentence = re.sub("[\n|\r]", " ", sentence)
  sentence = re.sub("[اِ|آ|إ|أ]", "ا", sentence)
  sentence = re.sub("[ة]", "ه", sentence)
  sentence = re.sub(r'[^0-9\u0600-\u06ff\u0750-\u077f\ufb50-\ufbc1\ufbd3-\ufd3f\ufd50-\ufd8f\ufd50-\ufd8f\ufe70-\ufefc\uFDF0-\uFDFD]+', ' ', sentence)
  return sentence

In [ ]:
from nltk.stem.isri import ISRIStemmer
from tashaphyne.stemming import ArabicLightStemmer
from nltk.tokenize import word_tokenize

def IsriStemmer(Document):
    #Initilizing object
    Stemmer = ISRIStemmer()
    StemedDoc = ""
    Words = word_tokenize(Document)
    for Word in Words:
        #Stemming everyword of the list
        StemedDoc = StemedDoc + Stemmer.stem(Word) + " "
    #Saving .. a new document built with the concatination of stemmed words
    return StemedDoc


def TashaRootStem(Document):
    #Initilizing object
    Stemmer = ArabicLightStemmer()
    StemText = ""
    #Tokenizing the document (every word will be stemmed alone)
    Words = word_tokenize(Document)
    for Word in Words:
        #first we initiate the word to a light stemme, then we get it's root
        Stemmer.light_stem(Word)
        StemText = StemText + Stemmer.get_root() + " "
    #Saving .. a new document built with the concatination of stemmed words
    return StemText

#Function transforming the sentence into a stemmed sentence
def TashaLightStem(Text):
    #Initilizing object
    Stemmer = ArabicLightStemmer()
    StemedText = ""
    Words = word_tokenize(Text)
    for Word in Words:
        #first we initiate the word to a light stemme, then we get it's stem
        Stemmer.light_stem(Word)
        StemedText = StemedText + Stemmer.get_stem() + " "

    #Saving .. a new document built with the concatination of stemmed words
    return StemedText



doc = "استحداث قوانين أخرى خاصة لضمان الحماية الجنائية للمعاملات الإلكترونية"
doc=clean_data(doc)
print(doc)
apply = TashaLightStem(doc)
print("light")
print(apply)
print("root")
apply2=TashaRootStem(doc)
print(apply2)
w="الاطفال"
w2="الطفل"
print(TashaRootStem(w2))
print(TashaRootStem(w))
print()
print(TashaLightStem(w2))
print(TashaLightStem(w))

استحداث قوانين اخرى خاصه لضمان الحمايه الجنائيه للمعاملات الالكترونيه
light
استحداث قوان خرى خاص ضما حمايه جنائيه معامل الالكترونيه 
root
حدث قنن خرر خوص لضم لحم جنء عمل للكترن 
طفل 
طفل 

طفل 
اطفال 


In [ ]:
TashaRootStem("مطره")

'مطر '

In [ ]:
TashaRootStem("الامطار")

'مطر '

In [ ]:
TashaRootStem("حدوثه")

'حدث '

In [ ]:
def remove_tashkeel(text):
    # Remove honorific sign
    text = text.replace("\u0610", "")  # ARABIC SIGN SALLALLAHOU ALAYHE WA SALLAM
    text = text.replace("\u0611", "")  # ARABIC SIGN ALAYHE ASSALLAM
    text = text.replace("\u0612", "")  # ARABIC SIGN RAHMATULLAH ALAYHE
    text = text.replace("\u0613", "")  # ARABIC SIGN RADI ALLAHOU ANHU
    text = text.replace("\u0614", "")  # ARABIC SIGN TAKHALLUS

    # Remove koranic anotation
    text = text.replace("\u0615", "")  # ARABIC SMALL HIGH TAH
    text = text.replace("\u0616", "")  # ARABIC SMALL HIGH LIGATURE ALEF WITH LAM WITH YEH
    text = text.replace("\u0617", "")  # ARABIC SMALL HIGH ZAIN
    text = text.replace("\u0618", "")  # ARABIC SMALL FATHA
    text = text.replace("\u0619", "")  # ARABIC SMALL DAMMA
    text = text.replace("\u061A", "")  # ARABIC SMALL KASRA
    text = text.replace("\u06D6", "")  # ARABIC SMALL HIGH LIGATURE SAD WITH LAM WITH ALEF MAKSURA
    text = text.replace("\u06D7", "")  # ARABIC SMALL HIGH LIGATURE QAF WITH LAM WITH ALEF MAKSURA
    text = text.replace("\u06D8", "")  # ARABIC SMALL HIGH MEEM INITIAL FORM
    text = text.replace("\u06D9", "")  # ARABIC SMALL HIGH LAM ALEF
    text = text.replace("\u06DA", "")  # ARABIC SMALL HIGH JEEM
    text = text.replace("\u06DB", "")  # ARABIC SMALL HIGH THREE DOTS
    text = text.replace("\u06DC", "")  # ARABIC SMALL HIGH SEEN
    text = text.replace("\u06DD", "")  # ARABIC END OF textH
    text = text.replace("\u06DE", "")  # ARABIC START OF RUB EL HIZB
    text = text.replace("\u06DF", "")  # ARABIC SMALL HIGH ROUNDED ZERO
    text = text.replace("\u06E0", "")  # ARABIC SMALL HIGH UPRIGHT RECTANGULAR ZERO
    text = text.replace("\u06E1", "")  # ARABICC SMALL HIGH DOTLESS HEAD OF KHAH
    text = text.replace("\u06E2", "")  # ARABIC SMALL HIGH MEEM ISOLATED FORM
    text = text.replace("\u06E3", "")  # ARABIC SMALL LOW SEEN
    text = text.replace("\u06E4", "")  # ARABIC SMALL HIGH MADDA
    text = text.replace("\u06E5", "")  # ARABIC SMALL WAW
    text = text.replace("\u06E6", "")  # ARABIC SMALL YEH
    text = text.replace("\u06E7", "")  # ARABIC SMALL HIGH YEH
    text = text.replace("\u06E8", "")  # ARABIC SMALL HIGH NOON
    text = text.replace("\u06E9", "")  # ARABIC PLACE OF SAJDAH
    text = text.replace("\u06EA", "")  # ARABIC EMPTY CENTRE LOW STOP
    text = text.replace("\u06EB", "")  # ARABIC EMPTY CENTRE HIGH STOP
    text = text.replace("\u06EC", "")  # ARABIC ROUNDED HIGH STOP WITH FILLED CENTRE
    text = text.replace("\u06ED", "")  # ARABIC SMALL LOW MEEM
    text = text.replace("\u0671", "\u0627")
    text = text.replace("\u0640", "") 

    #  Remove tashkeel
    text = text.replace("\u064B", "")  # ARABIC FATHATAN
    text = text.replace("\u064C", "")  # ARABIC DAMMATAN
    text = text.replace("\u064D", "")  # ARABIC KASRATAN
    text = text.replace("\u064E", "")  # ARABIC FATHA
    text = text.replace("\u064F", "")  # ARABIC DAMMA
    text = text.replace("\u0650", "")  # ARABIC KASRA
    text = text.replace("\u0651", "")  # ARABIC SHADDA
    text = text.replace("\u0652", "")  # ARABIC SUKUN
    text = text.replace("\u0653", "")  # ARABIC MADDAH ABOVE
    text = text.replace("\u0654", "")  # ARABIC HAMZA ABOVE
    text = text.replace("\u0655", "")  # ARABIC HAMZA BELOW
    text = text.replace("\u0656", "")  # ARABIC SUBSCRIPT ALEF
    text = text.replace("\u0657", "")  # ARABIC INVERTED DAMMA
    text = text.replace("\u0658", "")  # ARABIC MARK NOON GHUNNA
    text = text.replace("\u0659", "")  # ARABIC ZWARAKAY
    text = text.replace("\u065A", "")  # ARABIC VOWEL SIGN SMALL V ABOVE
    text = text.replace("\u065B", "")  # ARABIC VOWEL SIGN INVERTED SMALL V ABOVE
    text = text.replace("\u065C", "")  # ARABIC VOWEL SIGN DOT BELOW
    text = text.replace("\u065D", "")  # ARABIC REVERSED DAMMA
    text = text.replace("\u065E", "")  # ARABIC FATHA WITH TWO DOTS
    text = text.replace("\u065F", "")  # ARABIC WAVY HAMZA BELOW
    text = text.replace("\u0670", "")  # ARABIC LETTER SUPERSCRIPT ALEF
    text = text.replace("\u0622","ا")
    text = text.replace("\u0623","ا")
    text = text.replace("\u0625","ا")
    text = text.replace("\u0671","ا")
    text = text.replace("\u0672","ا")
    text = text.replace("\u0673","ا")
    return text

In [ ]:
def countNumber(sentence):
    count = 0
    for word in sentence:
        for character in word:
            if character.isdigit():
                count+=1
                break
    return count
def createBigramsList(terms):
  bigrams_list=[]
  bigram = ""
  b1=False
  b2=False
  terms_no_stop_words=[]
  for i in range(0,len(terms)):
    if terms[i] not in arb_stopwords:
      terms_no_stop_words.append(terms[i])
  
  for i in range(0,len(terms_no_stop_words)):
    if i+1<len(terms_no_stop_words):
      bigram = terms[i]+" "+terms[i+1]
      bigrams_list.append(bigram)
  return bigrams_list

def createTrigramsList(terms):
  trigrams_list=[]
  trgram = ""
  b1=False
  b2=False
  terms_no_stop_words=[]
  for i in range(0,len(terms)):
    if terms[i] not in arb_stopwords:
      terms_no_stop_words.append(terms[i])
  
  for i in range(0,len(terms_no_stop_words)):
    if i+2<len(terms_no_stop_words):
      trigram = terms[i]+" "+terms[i+1]+" "+terms[i+2]
      trigrams_list.append(trigram)
  return trigrams_list

# `Document Selector - TF-IDF Retrieval Model `

### Build TF-IDF Index

In [ ]:
words_dict={}
id_to_name={}
id=1
for i in range(1,27):
    with open('/content/Articles/'+str(i)+'_Article.txt', "r", encoding='utf-8') as ff:
            text=ff.read() #data.get_text()
            text=clean_data(text)
            text=remove_tashkeel(text)
            no_number_string = re.sub(r'\d+','',text)
            # remove all punctuation except words and space
            no_punc_string = re.sub(r'[^\w\s]','', no_number_string)
            # remove white spaces
            no_wspace_string = no_punc_string.strip()
            lst_string = [no_wspace_string][0].split()
            # remove stopwords
            no_stpwords_string=""
            for st in lst_string:
                if not st  in arb_stopwords:
                    no_stpwords_string += st+' '
            # removing last space
            #file.write(no_stpwords_string+ "\n")
            no_stpwords_string = no_stpwords_string[:-1]
            words = word_tokenize(no_stpwords_string)
            for word in words:
            #if word[0:4]!='http':  
                stemmed_word=TashaRootStem(word)
                if stemmed_word not in words_dict:
                       # wordsVsStem.update({stemmed_word:[word]})
                        lst=[1,[i]]
                        #wordsVsStem.update({word:stemmed_word})
                        words_dict.update({stemmed_word:lst})
                else:
                        #wlst=wordsVsStem.get(stemmed_word)
                        #if word not in wlst:
                        #    wlst.append(word)
                        #    wordsVsStem.update({stemmed_word:wlst})

                        lst = words_dict[stemmed_word]
                        if i not in lst[1]:
                            lst[0]=lst[0]+1
                            lst[1].append(i)
                            words_dict.update({stemmed_word:lst})
    id_to_name.update({i:str(i)+'_Article.txt'})
    #file.close()
# Serializing json 

# Serializing json 
json_object = json.dumps(id_to_name, indent = 2)
  
# Writing to sample.json
with open('/content/Indexes'+"/docID_to_name.txt","w", encoding="utf-8") as outfile:
    outfile.write(json_object)
outfile.close()
json_object = json.dumps(words_dict, indent = 4)
  
# Writing to sample.json
with open('/content/Indexes'+"/InvertedIndexBenchmark.txt","w", encoding="utf-8") as outfile:
    outfile.write(json_object)
outfile.close()
# Serializing json 
#json_object = json.dumps(wordsVsStem, indent = 4)
  
# Writing to sample.json
#with open(projectPath+"/WORDSvsSTEM_Benchmark.txt","w", encoding="utf-8") as outfile:
#    outfile.write(json_object)
#outfile.close()
sorted_word_dic={}
for i in sorted(words_dict):
   sorted_word_dic[i]=words_dict[i]
import json 
# Serializing json 
json_object = json.dumps(sorted_word_dic, indent = 4)
  
# Writing to sample.json
with open('/content/Indexes'+"/InvertedIndexBenchmark_sorted.txt","w", encoding="utf-8") as outfile:
    outfile.write(json_object)
outfile.close()

In [ ]:
with open('/content/Indexes/InvertedIndexBenchmark_sorted.txt', "r",encoding="utf-8") as json_file2:
    inverted_index = json.load(json_file2) 
with open('/content/Indexes/docID_to_name.txt') as json_file2:
    docId = json.load(json_file2) 

N=26
def tf_idf(doc,term, dft):
    tfidf=0
    file = open(doc, "r", encoding="utf-8")
    data = file.read()
    data=TashaRootStem(data)
    occurrences=data.count(term)
    if occurrences>0:
        tfidf = ( 1 + math.log2(occurrences) )*math.log2(N/dft)
    
    return tfidf


#Read the inverted index
def build_tfidf_index(inverted_index):
    words_dict={}
    for term, postinglst in inverted_index.items():
        docs_dict={}
        for doc in postinglst[1]:
            file_path="/content/Articles/"+str(docId.get(str(doc)))
            tfidf=tf_idf(file_path,term,postinglst[0])
            docs_dict.update({doc:tfidf})
        #sort each docs_dist based on the tfidf
        docs_list = sorted(docs_dict.items(), key=lambda x:x[1], reverse=True)
        sort_docs_dict = dict(docs_list)
        words_dict.update({term:sort_docs_dict})
    return words_dict

words_dict=build_tfidf_index(inverted_index)
json_object = json.dumps(words_dict, indent=2)
  
#Writing to sample.json
with open("/content/Indexes/TF-IDF_IndexBenchmark.txt","w", encoding="utf-8") as outfile:
    outfile.write(json_object)
outfile.close()

### TF-IDF Retrieval Model

In [ ]:
with open("/content/Indexes/TF-IDF_IndexBenchmark.txt", "r",encoding="utf-8") as json_file:
  tfidf_index = json.load(json_file)
with open("/content/Indexes/docID_to_name.txt") as json_file:
  docID_to_name = json.load(json_file)

In [ ]:
def selectDocument(query,tf_idf_index,docID_to_name):
    topDocs=[]
    plists=[]
    docs={}
    words=query.split()
    for word in words:
        pl=tfidf_index.get(TashaRootStem(word))
        plists.append(pl)
    for pl in plists:
        score=0
        if type(pl)==dict:
            for doc in pl:
                #if doc not in docs:
                    tfidf_sum=pl.get(doc)
                    for pl2 in plists:
                        if not (pl2==pl) and type(pl2)==dict:
                            if doc in pl2 :
                                tfidf_sum=tfidf_sum+pl2.get(doc)
                    docs.update({doc:tfidf_sum})
    docs_list = sorted(docs.items(), key=lambda x:x[1], reverse=True)
    sort_docs_dict = dict(docs_list)
    for x in list(sort_docs_dict)[0:6]:
        topDocs.append((x,  sort_docs_dict[x]))
    topDoc=topDocs[0][0]
    doc_name=docID_to_name.get(str(topDoc))
    doc_no=doc_name.split("_")[0]
    return (doc_name,doc_no)


# `Query Expansion `

### Generate Frequently Appearing Together Words

In [ ]:
import json
with open('/content/unique_words_frequency_.txt') as json_file:
    words_freq = json.load(json_file) 
with open('/content/unique_words_.txt') as json_file:
    unique_words = json.load(json_file) 

In [ ]:
import numpy as np
from scipy.sparse.linalg import svds
import json
import heapq
import re
import nltk
from nltk.stem.isri import ISRIStemmer
from matplotlib.pyplot import text 
import math
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('arabic'))
arb_stopwords = set(nltk.corpus.stopwords.words("arabic"))
words_freq_list = list(words_freq.items())

In [ ]:
def createMatrixA(m,n):
  A=[]
  for i in range(m):
    l=[0]*n
    l_final=[]
    if words_freq_list[i][0] in unique_words:
      l_final=unique_words.get(words_freq_list[i][0])
    for j in l_final:
      if j<n:
        l[j-1]=1
    A.append(l)
  return A

In [ ]:
k=6000
m=15000
A=createMatrixA(k,m)
U, S, Vt = np.linalg.svd(A)
C=np.matmul(np.matrix(U),np.matrix(U).transpose())

In [ ]:
dict_={}
added=[]
n=25
for row in range(0,k):
  print(row)
  c=np.array(C[row])[0]
  X=heapq.nlargest(n, range(len(c)), key=lambda x: c[x])
  for col in X:
    if row!=col and (col,row) not in added and (row,col) not in added:
      dict_.update({(words_freq_list[row][0],words_freq_list[col][0]):c[col]})
      added.append((row,col))

In [ ]:
docs_list = sorted(dict_.items(), key=lambda x:x[1], reverse=True)
sort_docs_dict = dict(docs_list)

In [ ]:
indices=[]
words_pair={}
i=0
while i<len(docs_list):
  if docs_list[i][0][0]!=docs_list[i][0][1] and docs_list[i][0][1] not in stop_words and docs_list[i][0][0] not in stop_words:
    
    if docs_list[i][0][0] not in words_pair:
      words_pair.update({docs_list[i][0][0].rstrip():[docs_list[i][0][1].rstrip()]})
    else:
      lst=words_pair.get(docs_list[i][0][0].rstrip())
      lst.append(docs_list[i][0][1].rstrip())
      words_pair.update({docs_list[i][0][0]:lst})
  i+=1

In [ ]:
json_object = json.dumps(words_pair)
  
with open('/content/DomainKnowledgeMaxScore25Words.txt',"w") as outfile:
    outfile.write(json_object)
outfile.close()

In [ ]:
with open("/content/___domainknowledge__.txt", "r", encoding='utf-8') as json_file:
  domain_knowledge_dict = json.load(json_file)

### Use Extracted Knowledge for Query Expansion

In [ ]:
import json
#with open('/content/DomainKnowledgeMaxScore25Words.txt') as json_file:
#    dn_bn = json.load(json_file) 
with open('/content/domainknowledgeMaxScores8K.txt') as json_file:
    dn = json.load(json_file) 

In [ ]:
def expand_query(query_terms_stemmed,dn):
  added_terms=[]
  for qt in query_terms_stemmed:
    if qt in dn:
      if len(dn.get(qt))>3:
        added_terms.append(dn.get(qt)[0])
        added_terms.append(dn.get(qt)[1])
        added_terms.append(dn.get(qt)[3])
      else:
        added_terms.append(dn.get(qt)[0])
  expanded_query=query_terms_stemmed+added_terms
  expanded_query_string=expanded_query[0]
  for i in range(1,len(expanded_query)):
    expanded_query_string=expanded_query_string+" "+expanded_query[i]
  return (expanded_query,expanded_query_string)

### Using Synonyms

In [ ]:
import json
with open('/content/unique_words_frequency.txt') as json_file:
    words_freq = json.load(json_file) 

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw')
nltk.download('omw-1.4')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


In [ ]:
def getSynonyms(word):
  synonyms=[]
  for synset in wn.synsets(word,lang=('arb')):
     for hypo in synset.hyponyms():
          for lemma in hypo.lemmas("arb"):
             synonyms.append(str(lemma).split('.')[3].replace("')",""))
  return synonyms 

In [ ]:
print(getSynonyms(''))

[]


In [ ]:
!pip install deep_translator

     |████████████████████████████████| 128 kB 10.2 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from deep_translator import GoogleTranslator

In [ ]:
translated = GoogleTranslator(source='ar', target='en').translate(text='مقبرة ')

In [ ]:
translated

'Cemetery'

In [ ]:
def en_to_ar_num(number_string):
    dic = {
        '0':'۰',
        '1':'١',
        '2':'٢',
        '3':'۳',
        '4':'٤',
        '5':'۵',
        '6':'٦',
        '7':'۷',
        '8':'۸',
        '9':'۹',
    }

    return "".join([dic[char] for char in number_string])

en_to_ar_num("945")

'۹٤۵'

In [ ]:
def containsNumber(value):
    indx=-1
    start=-1
    found=False
    for character in range(len(value)):
        if value[character].isdigit():
            if found==False:
                start = character
                found=True
            indx=character
    return (start,indx)

In [ ]:
def translate(txt,output):
    if output==False:
        translated_txt= GoogleTranslator(source='ar', target='en').translate(text=txt)
    elif output==True:
        translated_txt= GoogleTranslator(source='en', target='ar').translate(text=txt)
        #t=containsNumber(translated_txt)
        #num_in_txt=translated_txt[t[0]:t[1]+1]
        #tr_num_in_txt=en_to_ar_num(num_in_txt)
        #translated_txt=translated_txt[0:t[0]]+tr_num_in_txt+translated_txt[t[1]+1:len(translated_txt)]
    return translated_txt

In [ ]:
english_query =translate('اقدم',False)

In [ ]:
english_query

'old'

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
def get_synonyms(sentence):
  terms_list=sentence.split()
  syn = list()
  ant = list()
  new_sentence=''
  for term in terms_list:
      for synset in wordnet.synsets(term):
        for lemma in synset.lemmas():
          if lemma.name() not in syn:
            syn.append(lemma.name())    #add the synonyms
            new_sentence=new_sentence+' '+lemma.name()
            if lemma.antonyms():    #When antonyms are available, add them into the list
              ant.append(lemma.antonyms()[0].name())
  print('Synonyms: ' + str(syn))
  print('Antonyms: ' + str(ant))
  print(new_sentence)
  return new_sentence


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
syn_english_query=get_synonyms(english_query)

Synonyms: ['old', 'older', 'erstwhile', 'former', 'onetime', 'one-time', 'quondam', 'sometime', 'honest-to-god', 'honest-to-goodness', 'sure-enough', 'Old', 'previous']
Antonyms: []
 old older erstwhile former onetime one-time quondam sometime honest-to-god honest-to-goodness sure-enough Old previous


In [ ]:
syn_arabic_query=translate(syn_english_query,True)

In [ ]:
syn_arabic_query

'كبار السن السابق السابق مرة واحدة quondam في وقت ما صدق إلى الله الصدق إلى الخير متأكد بما فيه الكفاية قديم السابق'

In [ ]:
nltk.download('omw')
from nltk.corpus import wordnet as omw
jan = omw.synsets('oldest')[0]
print(jan)
print(jan.lemma_names(lang="eng"))
for i in jan.lemma_names(lang="eng"):
  print(translate(i,True))

[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Package omw is already up-to-date!
Synset('old.a.01')
['old']
عمر او قديم


# `Document Summarizer`
Graph based ranking model. Nodes of the graph are sentences. Query is a node in the graph as well. Compute the similarity between every sentence and the query. Compute the similarity between every two sentences. 

## Data Pre-processing 

Remove punctuation - tashkeel - stop words. Stemmed words version - Normal words Version. Create Bigrams - Trigrams - Unigrams --> for both article and query. Keep a mapping from orginal sentences to pre-processed ones.


In [ ]:
def query_preprocess(query):
  q_processed=remove_tashkeel(query)
  #remove punctuation
  q_processed=re.sub(r'[^\w\s]', '', q_processed)
  #remove stop words
  words_in_line=q_processed.split()
  q_no_stop_words=""
  for word in words_in_line:
    if len(q_no_stop_words)==0 and word not in arb_stopwords:
      q_no_stop_words=word
    elif word not in arb_stopwords:
      q_no_stop_words=q_no_stop_words+" "+word
  #stem words
  q_no_stop_words_list=q_no_stop_words.split()
  q_stem ="" 
  for word in q_no_stop_words_list:
    if len(q_stem)==0:
      q_stem=TashaRootStem(word)
    else:
      q_stem=q_stem+" "+TashaRootStem(word)
  return [q_processed,q_stem]


In [ ]:
def text_preprocess(text_file):
  st = ISRIStemmer()
  text_file_lines_n=text_file.split('.')
  text_file_lines_stemmed=[]
  text_file_lines=[]
  sentences_mapping_stemmed={}
  for Orgline in text_file_lines_n:
    line = clean_data(Orgline)
    org_line=line.replace("\n","")
    #remove tashkeel
    line_no_tashkeel=remove_tashkeel(line.replace("\n",""))
    #remove punctuation
    line_no_punct=re.sub(r'[^\w\s]', '', line_no_tashkeel)
    #remove stop words
    words_in_line=line_no_punct.split()
    line_no_stop_words=""
    for word in words_in_line:
      if len(line_no_stop_words)==0 and word not in arb_stopwords:
        line_no_stop_words=word
      elif word not in arb_stopwords:
        line_no_stop_words=line_no_stop_words+" "+word
    #stem words
    line_no_stop_words_list=line_no_stop_words.split()
    stemmed_line=""
    for word in line_no_stop_words_list:
      if len(stemmed_line)==0:
        stemmed_line=TashaRootStem(word)
      else:
        stemmed_line=stemmed_line+" "+TashaRootStem(word)
    if len(line_no_stop_words)>3:
      text_file_lines.append(line_no_stop_words)
    if len(stemmed_line)>3:
      text_file_lines_stemmed.append(stemmed_line)
      sentences_mapping_stemmed.update({stemmed_line:Orgline})
  return [text_file_lines,text_file_lines_stemmed,sentences_mapping_stemmed]


In [ ]:
def getWordsFrequency(lines):
  doc_words_freq_dict={}
  for line in lines:
    words=line.split()
    for word in words:
      if word not in doc_words_freq_dict:
        doc_words_freq_dict.update({word:1})
      else:
        c=doc_words_freq_dict.get(word)+1
        doc_words_freq_dict.update({word:c})
  return doc_words_freq_dict

## Summary Generation - Query-based & Non Query-based features




In [ ]:
def to_sentence_similarity(text_file, query,sentPos):
    scores=[0]*8
    doc=text_file
    sentences2=doc.split('.')
    query_terms=query.split()   
    query_terms_no_stop_words=[] 
    for term in query_terms:
      if term not in arb_stopwords:
        query_terms_no_stop_words.append(TashaRootStem(term))
    query_terms_bigrams=createBigramsList(query_terms)
    query_terms_trigrams=createTrigramsList(query_terms)
    weights2=[0.15,0.85,0.15,0.25,0.35,1,1,1]

    for t in range(1,len(text_file)):
        doc=doc+' '+text_file[t]
    #Separate article into sentences by full stop separation
    no_punct_doc = re.sub(r'[^\w\s]', '', doc)
    doc=remove_tashkeel(no_punct_doc)
    doc_words=TashaRootStem(doc).split()

    N_words_doc=len(doc_words)
    doc_words_freq_dict={}
    for word in doc_words:
      if TashaRootStem(word) not in doc_words_freq_dict and word not in arb_stopwords:
        doc_words_freq_dict.update({TashaRootStem(word):1})
      elif word not in arb_stopwords:
        c=doc_words_freq_dict.get(TashaRootStem(word))+1
        doc_words_freq_dict.update({TashaRootStem(word):c})
        

    sentences=doc.split('.')
    N=len(sentences)
    sentID=0
    final_scores=[0]*N
    #Save a dictionary that would map every sentence to its final sentence 
    sentences_score_dict={}
    noStopWordSentences={}
    noPunctuationSentences={}
    for sentence in sentences:
      no_punct_sent = re.sub(r'[^\w\s]', '', sentence)
      if sentence not in noPunctuationSentences:
        noPunctuationSentences.update({sentence:no_punct_sent})

      no_stop_words_sent=""
      no_punct_sent_list=no_punct_sent.split()
      for word in no_punct_sent_list:
        if word not in arb_stopwords and len(no_stop_words_sent)==0:
          no_stop_words_sent=word
        elif word not in arb_stopwords:
          no_stop_words_sent=no_stop_words_sent+" "+word
      if sentence not in noStopWordSentences:
        noStopWordSentences.update({sentence:[no_stop_words_sent,len(no_stop_words_sent.split())]})
    noStopWordsSentencesList = sorted(noStopWordSentences.items(), key=lambda x:x[1], reverse=True)
    sentence_with_max_len=noStopWordsSentencesList[0][1][1]
    id=0
    for sentence in sentences:
      if len(sentence)>10 and sentence!=query:
        #Tokenization --remove question marks, exclamation marks, and commas"
        no_punct_sent = noPunctuationSentences.get(sentence)

        #Sentence without stop words
        no_stop_words_sent=noStopWordSentences.get(sentence)[0]
        words = word_tokenize(no_stop_words_sent)

        #-----QUERY INDEPENDENT SCORES: 

        #Sentence Weight -- Sum of every word weight ---------------------------------------------->   1
        sentence_word_freq=0
        no_stop_words_sent_list=no_stop_words_sent.split()
        for word in words:
          stemmed_word=TashaRootStem(word)
          if TashaRootStem(word) in doc_words_freq_dict:
            sentence_word_freq+=doc_words_freq_dict.get(stemmed_word)
        if sentence_word_freq>0:
          scores[0]=len(no_stop_words_sent_list)/sentence_word_freq

        #Sentence Position Score ------------------------------------------------------------------>   2
        if sentPos!=sentences.index(sentence):
          scores[1]=1/abs(sentPos-sentences.index(sentence))

        #Sentence Length -------------------------------------------------------------------------->   3
        scores[2]=len(no_stop_words_sent_list)/sentence_with_max_len

        #-----QUERY DEPENDENT SCORES: 
        #Numeric Data ---------------------------------------------------------------------------->    4
        numerical_data_count=countNumber(no_stop_words_sent_list)
        if countNumber(query_terms)>0:
          numerical_data_count*=1.5
        if len(no_stop_words_sent_list)>0:
          scores[3]=numerical_data_count/len(no_stop_words_sent_list)

        #POS tagging score ----------------------------------------------------------------------->   5
        pos_tags=arabic_postagger.tag(sentence.split())
        proper_nouns_count=0
        proper_nouns_sentence_query_overlap=0
        for pos_tag in pos_tags:
          if pos_tag[1]=='NNP':
            proper_nouns_count+=1
          if pos_tag[1]=='NNP' and pos_tag[0] in query_terms:
            proper_nouns_sentence_query_overlap+=1
        if len(words)>0:
          scores[4]=(1.5*proper_nouns_sentence_query_overlap+proper_nouns_count)/len(words)

        #Sentence-Query Unigram Overlap
        sentence_query_unigram_overlap=0
        for term in query_terms_no_stop_words:
          if term in no_stop_words_sent_list:
            sentence_query_unigram_overlap+=1
        if len(query_terms)>0:
          scores[5]=sentence_query_unigram_overlap/len(query_terms)
        #Sentence-Query Bigram Overlap
        sentence_query_bigram_overlap=0
        sentence_bigrams=createBigramsList(no_stop_words_sent_list)
        for bigram in query_terms_bigrams:
          if bigram in sentence_bigrams:
            sentence_query_bigram_overlap+=1
        if len(query_terms_bigrams)>0:
          scores[6]=sentence_query_bigram_overlap/len(query_terms_bigrams)
        #Sentence-Query Trigram Overlap
        sentence_query_trigram_overlap=0
        sentence_trigrams=createTrigramsList(no_stop_words_sent_list)
        for trigram in query_terms_trigrams:
          if trigram in sentence_trigrams:
            sentence_query_trigram_overlap+=1
        if len(query_terms_trigrams)>0:
          scores[7]=sentence_query_trigram_overlap/len(query_terms_trigrams)
        final_score=0
        scores_list = [scores[i] * weights2[i] for i in range(len(scores))]
        for score in scores_list:
          final_score+=score
        sentID+=1
        sentences_score_dict.update({sentences2[id]:[final_score,sentID]})
        id+=1

        

    sentences_query_score_dict_sorted = sorted(sentences_score_dict.items(), key=lambda x:x[1], reverse=True)
    return sentences_query_score_dict_sorted
   

In [ ]:
def sentence_query_similarity(text_file, query):
    weights=[0,0,0,0.15,0.55,1.5,2.5,0.6,3]
    scores=[0]*9
    doc=text_file[0]
    query_terms=query.split()   
    query_terms_no_stop_words=[] 
    for term in query_terms:
      if term not in arb_stopwords:
        query_terms_no_stop_words.append(term)
    query_terms_bigrams=createBigramsList(query_terms)
    query_terms_trigrams=createTrigramsList(query_terms)
    query_terms_stemmed=[]
    for term in query_terms:
      if term not in arb_stopwords:
        query_terms_stemmed.append(st.stem(term))    

    for t in range(1,len(text_file)):
        doc=doc+' '+text_file[t]
    #Separate article into sentences by full stop separation
    no_punct_doc = re.sub(r'[^\w\s]', '', doc)
    doc=remove_tashkeel(doc)
    doc_words=no_punct_doc.split()

    N_words_doc=len(doc_words)
    doc_words_freq_dict={}
    for word in doc_words:
      if word not in doc_words_freq_dict and word not in arb_stopwords:
        doc_words_freq_dict.update({word:1})
      elif word not in arb_stopwords:
        c=doc_words_freq_dict.get(word)+1
        doc_words_freq_dict.update({word:c})
        

    sentences=doc.split('.')
    N=len(sentences)
    sentID=0
    final_scores=[0]*N
    #Save a dictionary that would map every sentence to its final sentence 
    sentences_score_dict={}
    noStopWordSentences={}
    noPunctuationSentences={}
    for sentence in sentences:
      no_punct_sent = re.sub(r'[^\w\s]', '', sentence)
      if sentence not in noPunctuationSentences:
        noPunctuationSentences.update({sentence:no_punct_sent})

      no_stop_words_sent=""
      no_punct_sent_list=no_punct_sent.split()
      for word in no_punct_sent_list:
        if word not in arb_stopwords and len(no_stop_words_sent)==0:
          no_stop_words_sent=word
        elif word not in arb_stopwords:
          no_stop_words_sent=no_stop_words_sent+" "+word
      if sentence not in noStopWordSentences:
        noStopWordSentences.update({sentence:[no_stop_words_sent,len(no_stop_words_sent.split())]})
    noStopWordsSentencesList = sorted(noStopWordSentences.items(), key=lambda x:x[1], reverse=True)
    sentence_with_max_len=noStopWordsSentencesList[0][1][1]
    for sentence in sentences:
      if len(sentence)>10:
        #Tokenization --remove question marks, exclamation marks, and commas"
        no_punct_sent = noPunctuationSentences.get(sentence)

        #Sentence without stop words
        no_stop_words_sent=noStopWordSentences.get(sentence)[0]
        words = word_tokenize(no_stop_words_sent)

        #-----QUERY INDEPENDENT SCORES: 

        #Sentence Weight -- Sum of every word weight ---------------------------------------------->   1
        sentence_word_freq=0
        no_stop_words_sent_list=no_stop_words_sent.split()
        query_specific_sentence_weight=0
        for word in words:
          #stemmed_word=st.stem(word)
          if word in doc_words_freq_dict:
            sentence_word_freq+=doc_words_freq_dict.get(word)

          if st.stem(word) in query_terms_stemmed:
            query_specific_sentence_weight+=1
        scores[0]=len(no_stop_words_sent_list)/sentence_word_freq
        if query_specific_sentence_weight>0:
          scores[7]=len(no_stop_words_sent_list)/query_specific_sentence_weight
        #Sentence Position Score ------------------------------------------------------------------>   2
        scores[1]=(N-sentences.index(sentence)+1)/N

        #Sentence Length -------------------------------------------------------------------------->   3
        scores[2]=len(no_stop_words_sent_list)/sentence_with_max_len

        #-----QUERY DEPENDENT SCORES: 
        #Numeric Data ---------------------------------------------------------------------------->    4
        numerical_data_count=countNumber(no_stop_words_sent_list)
        if countNumber(query_terms)>0:
          numerical_data_count*=1.5
        if len(no_stop_words_sent_list)>0:
          scores[3]=numerical_data_count/len(no_stop_words_sent_list)

        #POS tagging score ----------------------------------------------------------------------->   5
        pos_tags=arabic_postagger.tag(sentence.split())
        proper_nouns_count=0
        proper_nouns_sentence_query_overlap=0
        for pos_tag in pos_tags:
          #if pos_tag[1]=='NNP' or pos_tag[1]=='VBD':
          #  proper_nouns_count+=0.25
          if (pos_tag[1]=='NNP' or pos_tag[1]=='VBD') and pos_tag[0] in query_terms:
            proper_nouns_sentence_query_overlap+=1.25
          scores[4]=(1.5*proper_nouns_sentence_query_overlap)

        #Sentence-Query Unigram Overlap
        sentence_query_unigram_overlap=0
        for term in query_terms_no_stop_words:
          if term in no_stop_words_sent_list:
            sentence_query_unigram_overlap+=(1)
        if len(query_terms)>0:
          scores[5]=sentence_query_unigram_overlap/len(query_terms)
        #Sentence-Query Bigram Overlap
        sentence_query_bigram_overlap=0
        sentence_bigrams=createBigramsList(no_stop_words_sent_list)
        for bigram in query_terms_bigrams:
          if bigram in sentence_bigrams:
            sentence_query_bigram_overlap+=1
            unigrams=bigram.split()
            #print(unigrams)
            #if abs(doc_words_freq_dict.get(unigrams[0])-doc_words_freq_dict.get(unigrams[1]))>2:
            #  sentence_query_bigram_overlap*=1.5
            #print("YEEEESSS")
          
        if len(query_terms_bigrams)>0:
          scores[6]=sentence_query_bigram_overlap/len(query_terms_bigrams)
        #Sentence-Query Bigram Overlap
        sentence_query_trigram_overlap=0
        sentence_trigrams=createTrigramsList(no_stop_words_sent_list)
        for trigram in query_terms_trigrams:
          if trigram in sentence_trigrams:
            sentence_query_trigram_overlap+=1
        if len(query_terms_trigrams)>0:
          scores[8]=sentence_query_trigram_overlap/len(query_terms_trigrams)
        final_score=0
        scores_list = [scores[i] * weights[i] for i in range(len(scores))]
        for score in scores_list:
          final_score+=score
        sentID+=1
        sentences_score_dict.update({sentence:[final_score,sentID]})

        

    sentences_query_score_dict_sorted = sorted(sentences_score_dict.items(), key=lambda x:x[1], reverse=True)
    return sentences_query_score_dict_sorted
    #top_ranked_summary_sentences={}
    #for i in range(0,int(N*0.1)):
    #  top_ranked_summary_sentences.update({sentences_score_dict_sorted[i][0]:sentences_score_dict_sorted[i][1][1]})
    #top_ranked_summary_sentences_sorted = sorted(top_ranked_summary_sentences.items(), key=lambda x:x[1])
    #print(sentences_score_dict_sorted)
    #final_summary=""
    #for sentence in top_ranked_summary_sentences_sorted:
    #  final_summary=final_summary+" "+sentence[0]
    #print()
    #print('---------- Summary ----------')
    #print(final_summary)

In [ ]:
import math
def sentence_sentence_similarity(text_file,query):
    doc=text_file[0]
    for t in range(1,len(text_file)):
        doc=doc+' '+text_file[t]

    #Separate article into sentences by full stop separation
    no_punct_doc = re.sub(r'[^\w\s]', '', doc)
    doc=remove_tashkeel(no_punct_doc)
    #print(doc)
    
    doc_words=doc.split()
    query=remove_tashkeel(query)

    N_words_doc=len(doc_words)
    doc_words_freq_dict={}

    for word in doc_words:
      if st.stem(word) not in doc_words_freq_dict and word not in arb_stopwords:
        doc_words_freq_dict.update({st.stem(word):1})
      elif word not in arb_stopwords:
        c=doc_words_freq_dict.get(st.stem(word))+1
        doc_words_freq_dict.update({st.stem(word):c})
        

    sentences=doc.split('.')
    N=len(sentences)
    sentPos=0
    sentID=0
    final_scores=[0]*N
    #sentences_to_sentences_dict={}
    #for sentence in sentences:
    #  this_sentence_to_all_sentences_sim = to_sentence_similarity(text_file,sentence,sentPos)
    #  sentPos+=1
    #  if sentence not in sentences_to_sentences_dict:
    #    sentences_to_sentences_dict.update({sentence:dict(this_sentence_to_all_sentences_sim)})
    sentences_query_similarity=sentence_query_similarity(text_file,query)
    sentences_query_similarity_dict=dict(sentences_query_similarity)
    sentences_aggregated_scores={}
    sentence_with_max_to_query_sim=sentences_query_similarity[0][0]
    sentence_with_2ndmax_to_query_sim=sentences_query_similarity[1][0]
    sentences_query_similarity2=to_sentence_similarity(text_file,sentence_with_max_to_query_sim, sentences_query_similarity[0][1][1])
    sentences_query_similarity3=to_sentence_similarity(text_file,sentence_with_2ndmax_to_query_sim, sentences_query_similarity[1][1][1])
    sentences_query_similarity_dict2=dict(sentences_query_similarity2)
    sentences_query_similarity_dict3=dict(sentences_query_similarity3)
    sentence_scores={}
    for sentence in sentences:
      if sentence!='\n' and len(sentence)>3:
        if sentence in sentences_query_similarity_dict and sentence in sentences_query_similarity_dict2 and  sentence!=sentence_with_max_to_query_sim and sentence in sentences_query_similarity_dict3 and  sentence!=sentence_with_2ndmax_to_query_sim:
          #print(sentence)
          #print(sentences_query_similarity_dict.get(sentence)[0])
          #print(sentences_query_similarity_dict2.get(sentence)[0])
          sentence_agg_sc=sentences_query_similarity_dict.get(sentence)[0]+sentences_query_similarity_dict2.get(sentence)[0]
          sentence_scores.update({sentence:[sentence_agg_sc,sentences_query_similarity_dict.get(sentence)[1]]})
        elif sentence in sentences_query_similarity_dict:
          #print(sentence)
          sentence_agg_sc=sentences_query_similarity_dict.get(sentence)[0]
          sentence_scores.update({sentence:[sentence_agg_sc,sentences_query_similarity_dict.get(sentence)[1]]})


    sentences_score_dict_sorted = sorted(sentence_scores.items(), key=lambda x:x[1], reverse=True)
    top_ranked_summary_sentences={}
    if N<20:
      bound = math.ceil(0.3*N)
    else:
      bound = math.ceil(0.25*N)
    for i in range(0,bound):
      top_ranked_summary_sentences.update({sentences_score_dict_sorted[i][0]:sentences_score_dict_sorted[i][1][1]})
    top_ranked_summary_sentences_sorted = sorted(top_ranked_summary_sentences.items(), key=lambda x:x[1])
    print(sentences_score_dict_sorted)
    final_summary=""
    for sentence in top_ranked_summary_sentences_sorted:
      final_summary=final_summary+" "+sentence[0]
    print()
    print('---------- Summary ----------')
    print(final_summary)
    return final_summary



In [ ]:
text = "وَمِنَ النَّاسِ مَنْ يَقُولُ آمَنَّا بِاللَّهِ وَبِالْيَوْمِ الآخِرِ وَمَا هُمْ بِمُؤْمِنِينَ "

In [ ]:
clean_data(text)

'ومن الناس من يقول امنا بالله وباليوم الاخر وما هم بمؤمنين '

## Syntaxic Similarity Measures

In [ ]:
def SimilarityOrder(l1, l2):
    p1 = l1.split()  # sentence 1 with tockens that match
    p2 = l2.split()  # sentence 2 with tockens that match
    m = len(p1)
    n = len(p2)
    k = list(p1)
    g = list(p2)
    listeMax = []
    a1 = []
    a2 = []
    s = 0

    for i in range(len(p1)):
        if (len(p1) < i):
            break
        for j in range(len(p2)):
            if p1[i] == p2[j]:
                a1.append(p1[i])
                s = s + 1
                del p2[j]
                break
    for i in range(len(g)):
        if (len(g) < i):
            break
        for j in range(len(k)):
            if g[i] == k[j]:
                a2.append(g[i])
                del k[j]
                break

    d = list(a1)
    for i in range(len(a1)):
        a1[i] = i
    for i in range(len(d)):
        for j in range(len(a2)):
            if d[i] == a2[j]:
                a2[j] = i
    if s > 0:
        f = [abs(a2_elt - a1_elt) for a2_elt, a1_elt in
             zip(a2, a1)]
        somme = sum(f)

        if s > 1 and (s % 2 != 0):
            so = 1 - ((2 * somme) / (pow(s, 2) - 1))
        elif s == 1 and (s % 2 != 0):
            so = 1
        elif s % 2 == 0 and s > 0:
            so = 1 - ((2 * somme) / pow(s, 2))
    elif s == 0:
        so = 0
    return so

### Common Words

In [ ]:
def commonWord(sentence1, sentence2):
    sent1 = sentence1.split()  
    sent2 = sentence2.split()  
    list1 = list(sent1)
    list2 = list(sent2)
    data1 = []
    data2 = []
    s = 0

    for i in range(len(sent1)):
        if (len(sent1) < i):
            break
        for j in range(len(sent2)):
            if sent1[i] == sent2[j]:
                data1.append(sent1[i])
                s = s + 1
                del sent2[j]
                break
    print(data1)
    for i in range(len(list2)):
        if (len(list2) < i):
            break
        for j in range(len(list1)):
            if list2[i] == list1[j]:
                data2.append(list2[i])
                del list1[j]
                break

    print(data2)
    d = list(data1)
    for i in range(len(data1)):
        data1[i] = i 
    for i in range(len(d)):
        for j in range(len(data2)):
            if d[i] == data2[j]:
                data2[j] = i
    if s > 0:  
        f = [abs(data2_elem - data1_elem) for data2_elem, data1_elem in
             zip(data2, data1)] 
        somme = sum(f)
        if s > 1 and (s % 2 != 0):
            sim = 1 - ((2 * somme) / (pow(s, 2) - 1))
        elif s == 1 and (s % 2 != 0):
            sim = 1
        elif s % 2 == 0 and s > 0:
            sim = 1 - ((2 * somme) / pow(s, 2))
    elif s == 0:
        sim = 0
    print(sim)
    return sim

In [ ]:
doc = "فرع من فروع التكنولوجيا المعروفة باسم أمن المعلومات"
doc2 = "فرع من فروع التكنولوجيا أمن المعلومات"
commonWord(doc,doc2)

['فرع', 'من', 'فروع', 'التكنولوجيا', 'أمن', 'المعلومات']
['فرع', 'من', 'فروع', 'التكنولوجيا', 'أمن', 'المعلومات']
1.0


1.0

### Manhatten Distance

In [ ]:
from math import *
import re, math
from collections import Counter

WORD = re.compile(r'\w+')


def get_manhattan(vec1, vec2):
    return sum(abs(vec1.get(k, 0) - vec2.get(k, 0)) for k in set(vec1.keys()).union(set(vec2.keys())))


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)


def manhattan(text1, text2):
    vector1 = text_to_vector(text1)

    vector2 = text_to_vector(text2)

    manhattan = get_manhattan(vector1, vector2)
    return abs(1 - (manhattan / max(len(vector1), len(vector2))))



vec1 = "السماء صافية اليوم"
vec2 =  "صافية اليوم"
apply1 = manhattan(doc, vec2)
apply = manhattan(doc, doc2)
print(apply1)
print(apply)

0.25
0.75


### Euclidean Distance

In [ ]:
import re, math
from collections import Counter
from math import *
import codecs as c

WORD = re.compile(r'\w+')


def get_euclidean(vec1, vec2):
    return sqrt(
        sum((vec1.get(k, 0) - vec2.get(k, 0)) ** 2 for k in set(vec1.keys()).union(set(vec2.keys()))))


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)


def euclidien(text1, text2):
    vector1 = text_to_vector(text1)
    vector2 = text_to_vector(text2)
    euclid_dist = get_euclidean(vector1, vector2)

    return 1 - (euclid_dist / max(len(vector1), len(vector2)))
print(euclidien(vec1,vec2))
print(euclidien(doc,vec2))

0.6666666666666667
0.6047152924789525


### Jaccard Similarity

In [ ]:
def jaccard(sentence1, sentence2):
    intersection_cardinality = len(set.intersection(*[set(sentence1), set(sentence2)]))
    union_cardinality = len(set.union(*[set(sentence1), set(sentence2)]))

    return intersection_cardinality / float(union_cardinality)
print(jaccard(vec1,vec2))
print(jaccard(doc,vec2))

0.8181818181818182
0.4444444444444444


### Jaro Similarity

In [ ]:
def jaro(s, t):
    s_len = len(s)
    t_len = len(t)

    if s_len == 0 and t_len == 0:
        return 1

    match_distance = (max(s_len, t_len) // 2) - 1

    s_matches = [False] * s_len
    t_matches = [False] * t_len

    matches = 0
    transpositions = 0

    for i in range(s_len):
        start = max(0, i - match_distance)
        end = min(i + match_distance + 1, t_len)

        for j in range(start, end):
            if t_matches[j]:
                continue
            if s[i] != t[j]:
                continue
            s_matches[i] = True
            t_matches[j] = True
            matches += 1
            break

    if matches == 0:
        return 0

    k = 0
    for i in range(s_len):
        if not s_matches[i]:
            continue
        while not t_matches[k]:
            k += 1
        if s[i] != t[k]:
            transpositions += 1
        k += 1

    return ((matches / s_len) +
            (matches / t_len) +
            ((matches - transpositions / 2) / matches)) / 3


In [ ]:
print(jaro(vec1,vec2))
print(jaro(doc,vec2))

0.7037037037037037
0.5030451574569222


### Levenshtein Distance

In [ ]:
def levenshtein(s1, s2):
    d = {}
    lenstr1 = len(s1)
    lenstr2 = len(s2)
    for i in range(-1, lenstr1 + 1):
        d[(i, -1)] = i + 1
    for j in range(-1, lenstr2 + 1):
        d[(-1, j)] = j + 1

    for i in range(lenstr1):
        for j in range(lenstr2):
            if s1[i] == s2[j]:
                cost = 0
            else:
                cost = 1
            d[(i, j)] = min(
                d[(i - 1, j)] + 1,  # deletion
                d[(i, j - 1)] + 1,  # insertion
                d[(i - 1, j - 1)] + cost,  # substitution
            )
            if i and j and s1[i] == s2[j - 1] and s1[i - 1] == s2[j]:
                d[(i, j)] = min(d[(i, j)], d[i - 2, j - 2] + cost)  # transposition

    return 1 - (d[lenstr1 - 1, lenstr2 - 1] / max(lenstr1, lenstr2))

levenshtein(vec1,vec2)

0.6111111111111112

In [ ]:
levenshtein(doc,vec2)

0.1568627450980392

In [ ]:
#the smaller the value returned, the better
def levenshteinOcc(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1
    distances = range(len(s1) + 1) 
    print(distances)
    for index2,char2 in enumerate(s2):
        newDistances = [index2+1]
        for index1,char1 in enumerate(s1):
            if char1 == char2:
                newDistances.append(distances[index1]) 
            else:
                 newDistances.append(1 + min((distances[index1], distances[index1+1], newDistances[-1]))) 
        distances = newDistances 
    return distances[-1]
x='توابيت'
x2='تابوت'
print(levenshteinOcc(x,x2))
#print(levenshtein("مرحبا بكم بيننا","السلام عليكم"))

range(0, 6)
2


### Dice Distance

In [ ]:
def dice(a, b):
    if not len(a) or not len(b): return 0.0
    if len(a) == 1:  a = a + u'.'
    if len(b) == 1:  b = b + u'.'

    a_bigram_list = []
    for i in range(len(a) - 1):
        a_bigram_list.append(a[i:i + 2])
    b_bigram_list = []
    for i in range(len(b) - 1):
        b_bigram_list.append(b[i:i + 2])

    a_bigrams = set(a_bigram_list)
    b_bigrams = set(b_bigram_list)
    overlap = len(a_bigrams & b_bigrams)

    dice_coeff = overlap * 2.0 / (len(a_bigrams) + len(b_bigrams))

    return dice_coeff

In [ ]:
dice(doc,vec2)

0.16666666666666666

In [ ]:
dice(vec1,vec2)

0.7692307692307693

### LCS Distance 

In [ ]:
def LCS(s1, s2):
    if (len(s1) != 0 and len(s2) != 0):
        matrix = [[0 for x in range(len(s2))] for x in range(len(s1))]
        cs = ""
        for i in range(len(s1)):
            for j in range(len(s2)):
                if s1[i] == s2[j]:
                    if i == 0 or j == 0:
                        matrix[i][j] = 1
                        cs += s1[i]
                    else:
                        matrix[i][j] = matrix[i - 1][j - 1] + 1
                        cs += s1[i]
                else:
                    if i == 0 or j == 0:
                        matrix[i][j] = 0
                    else:
                        matrix[i][j] = max(matrix[i - 1][j], matrix[i][j - 1])

        return matrix[len(s1) - 1][len(s2) - 1] / max(len(s1), len(s2))
    else:

        return 0

In [ ]:
LCS(vec1,vec2)

0.6111111111111112

In [ ]:
LCS(doc,vec2)

0.1568627450980392

### Similarity Score

In [ ]:
print(LCS(vec1,vec2))
print(jaro(vec1,vec2))
print(dice(vec1,vec2))
print(jaccard(vec1,vec2))
print(levenshtein(vec1,vec2))
print(manhattan(vec1,vec2))
print(euclidien(vec1,vec2))

0.6111111111111112
0.7037037037037037
0.7692307692307693
0.8181818181818182
0.6111111111111112
0.6666666666666667
0.6666666666666667


In [ ]:
print(LCS(doc,doc2))
print(jaro(doc,doc2))
print(dice(doc,doc2))
print(jaccard(doc,doc2))
print(levenshtein(doc,doc2))
print(manhattan(doc,doc2))
print(euclidien(doc,doc2))

0.7254901960784313
0.8544426779720897
0.8656716417910447
0.8235294117647058
0.7254901960784313
0.75
0.8232233047033631


In [ ]:
!pip3 install textdistance

In [ ]:
import textdistance
textdistance.ratcliff_obershelp(doc,doc2)


0.8409090909090909

In [ ]:
textdistance.jaccard(doc,doc2)

0.7254901960784313

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
def getOccurences(sentence):    ##### Occurrence calculation
    try:
        vec = CountVectorizer()
        X = vec.fit_transform([sentence]).toarray()
        lengthSentence = np.sum(X[0])
        occurenceGreatToOne = []
        for i in X[0]:
            if i > 1:
                occurenceGreatToOne.append(i)
    except:
        occurenceGreatToOne=0
        lengthSentence=1
    return occurenceGreatToOne,lengthSentence

In [ ]:
def getSyntaxicSim2(sen1,sen2):
  #weights=[0.6,0.55,0.75,0.95,0.7,0.95,0.6]
  weights=[0.6,0.55,0.75,0.95,0.95,0.5,0.75]
  score=0
  score=weights[0]*textdistance.ratcliff_obershelp(sen1,sen2)#manhattan(sen1,sen2)
  score+=weights[1]*textdistance.arith_ncd(sen1,sen2)
  #Token based
  score+=weights[2]*textdistance.jaccard(sen1,sen2)
  score+=weights[3]*textdistance.jaro_winkler(sen1,sen2)
  score+=weights[4]*textdistance.levenshtein(sen1,sen2)#levenshtein(sen1,sen2)
  #Edit based
  score+=weights[5]*textdistance.hamming.normalized_similarity(sen1,sen2)#dice(sen1,sen2)
  #Sequence based
  score+=weights[6]*textdistance.tanimoto(sen1,sen2)

  return score

In [ ]:
textdistance.tanimoto(vec1,doc2)

-1.691877704637668

In [ ]:
def getSyntaxicSim(sen1,sen2):
  weights=[0.7,0.55,0.75,0.95,0.7,0.95,0.85]
  score=weights[0]*manhattan(sen1,sen2)
  #score+=weights[1]*euclidien(sen1,sen2)
  score=weights[2]*jaccard(sen1,sen2)
  score+=weights[3]*jaro(sen1,sen2)
  #score+=weights[4]*levenshtein(sen1,sen2)
  #score+=weights[5]*SimilarityOrder(sen1,sen2)#dice(sen1,sen2)
  score+=weights[6]*LCS(sen1,sen2)

  return score

## Summary Generation - Query-based features

In [ ]:
def generate_summary(query,tf_idf_index,docID_to_name):
  #Read Article 
  doc=selectDocument(query,tf_idf_index,docID_to_name)
  print("Currently reading document: "+doc[0]+" for query: "+query)
  i=doc[1]
  with open('/content/Articles/'+str(i)+'_Article.txt', "r", encoding='utf-8') as ff:
    text=ff.read() 
  #----------------------------------------------ARTICLE--------------------------------------------
  text_preproc=text_preprocess(text)
  lines=text_preproc[0]
  map=text_preproc[2]
  lines_stemmed=text_preproc[1]
  N=len(lines)
  doc_words_freq_dict={}
  for line in lines_stemmed:
    words=line.split()
    for word in words:
      if word not in doc_words_freq_dict:
        doc_words_freq_dict.update({word:1})
      else:
        c=doc_words_freq_dict.get(word)+1
        doc_words_freq_dict.update({word:c})
  
  #-----------------------------------------------QUERY---------------------------------------------
  query_preproc=query_preprocess(query)
  query_terms_no_stop_words=query_preproc[0].split()
  #expanded_query=expand_query(query_preproc[1].split(),dn)
  query_terms_stemmed=query_preproc[1].split()
  query_terms_bigrams=createBigramsList(query_terms_no_stop_words)
  query_terms_bigrams_stem=createBigramsList(query_terms_stemmed)

  scores=[0]*5
  #weights=[0.65,0.75,0.95,0.60,0.95]
  #weights=[0.85,0.55,0.95,0.35,0.85]
  #weights=[0.90,0.65,0.85,0.85,0.75]
  #weights=[0.85,0.75,0.45,0.75,0.65]
  #weights=[0.75,0.95,0.35,0.55,0.55]
  #weights=[0.65,0.85,0.25,0.75,0.45]
  weights=[1,0.75,0,0.650,1]
  #----------------------------------- -- FEATURES EXTRACTION -- -----------------------------------
  sentences_score_dict={}
  for sentID in range(0,len(lines)):
    sentence_words=lines[sentID].split()
    sentence_stemmed_words=lines_stemmed[sentID].split()

    #EXTRACT QUERY RELATED FEATURES

    # ---- 1 ---- QUERY SENTENCE UNIGRAM OVERLAP
    query_sentence_unigram_overlap=0
    for wordID in range(0,len(query_terms_stemmed)):
      if query_terms_stemmed[wordID] in sentence_stemmed_words:
        query_sentence_unigram_overlap+=1
      #if query_terms_no_stop_words[wordID] in sentence_words:
      #  query_sentence_unigram_overlap+=1
    scores[0]=query_sentence_unigram_overlap
    
    # ---- 2 ---- QUERY SENTENCE BIGRAM OVERLAP
    query_sentence_bigram_overlap=0
    sentence_bigrams_stemmed=createBigramsList(sentence_stemmed_words)
    sentence_bigrams=createBigramsList(sentence_words)
    for wordID in range(0,len(query_terms_bigrams_stem)):
      if query_terms_bigrams_stem[wordID] in sentence_bigrams_stemmed:
        query_sentence_bigram_overlap+=1
      #if query_terms_bigrams[wordID] in sentence_bigrams:
      #  query_sentence_bigram_overlap+=1
    scores[1]=query_sentence_bigram_overlap

    # ---- 3 ---- QUERY SENTENCE TRIGRAM OVERLAP
    #query_sentence_trigram_overlap=0
    #sentence_trigrams_stemmed=createTrigramsList(sentence_stemmed_words)
    #sentence_trigrams=createBigramsList(sentence_words)
    #query_terms_trigrams=createBigramsList(query_terms_no_stop_words)
    #query_terms_trigrams_stem=createBigramsList(query_terms_stemmed)
    #for wordID in range(0,len(query_terms_trigrams_stem)):
    #  if query_terms_trigrams_stem[wordID] in sentence_trigrams_stemmed:
    #    query_sentence_trigram_overlap+=1
      #if query_terms_trigrams[wordID] in sentence_trigrams:
      #  query_sentence_trigram_overlap+=1
    #scores[2]=query_sentence_trigram_overlap
    
    # ---- 4 ---- POS tagging with query comparision
    pos_tags=arabic_postagger.tag(sentence_words)
    proper_nouns_count=0
    proper_nouns_sentence_query_overlap=0
    for pos_tag in pos_tags:
      #if (pos_tag[1]=='NNP' or pos_tag[1]=='VBD') and pos_tag[0] in query_terms_no_stop_words:
      if pos_tag[1]=='NNP' and pos_tag[0] in query_terms_no_stop_words:
        proper_nouns_sentence_query_overlap+=1.25
      #if pos_tag[1]=='NNP':
      #  proper_nouns_sentence_query_overlap+=1
    scores[3]=proper_nouns_sentence_query_overlap/len(sentence_words)
    
    # ---- 5 ---- Syntaxic simlarity between sentence and query
    syntaxic_sim=getSyntaxicSim(lines_stemmed[sentID],query_preproc[1])
    scores[4]=syntaxic_sim
    
    final_score=0
    scores_list = [scores[i] * weights[i] for i in range(len(scores))]
    for score in scores_list:
      final_score+=score
    sentences_score_dict.update({lines_stemmed[sentID]:[final_score,sentID]})

  sentences_score_dict_sorted = sorted(sentences_score_dict.items(), key=lambda x:x[1], reverse=True)
  top_ranked_summary_sentences={}
  if N<20:
    bound = math.ceil(0.3*N) #0.35
    
  else:
    bound = math.ceil(0.2*N) #0.25
    
  for i in range(0,bound):
    top_ranked_summary_sentences.update({sentences_score_dict_sorted[i][0]:sentences_score_dict_sorted[i][1][1]})
  top_ranked_summary_sentences_sorted = sorted(top_ranked_summary_sentences.items(), key=lambda x:x[1])
  #print(sentences_score_dict_sorted)
  final_summary=""
  for sentence in top_ranked_summary_sentences_sorted:
    final_summary=final_summary+" "+map.get(sentence[0])
  print()
  print('------------------------------------------------------------- Summary -------------------------------------------------------------')
  print()
  print(final_summary)
  print()
  return final_summary


In [ ]:
def generate_summary1(query,tf_idf_index,docID_to_name):
  #Read Article 
  doc=selectDocument(query,tf_idf_index,docID_to_name)
  print("Currently reading document: "+doc[0]+" for query: "+query)
  i=doc[1]
  with open('/content/Articles/'+str(i)+'_Article.txt', "r", encoding='utf-8') as ff:
    text=ff.read() 
  #----------------------------------------------ARTICLE--------------------------------------------
  text_preproc=text_preprocess(text)
  lines=text_preproc[0]
  map=text_preproc[2]
  lines_stemmed=text_preproc[1]
  N=len(lines)
  doc_words_freq_dict={}
  for line in lines_stemmed:
    words=line.split()
    for word in words:
      if word not in doc_words_freq_dict:
        doc_words_freq_dict.update({word:1})
      else:
        c=doc_words_freq_dict.get(word)+1
        doc_words_freq_dict.update({word:c})
  
  #-----------------------------------------------QUERY---------------------------------------------
  query_preproc=query_preprocess(query)
  query_terms_no_stop_words=query_preproc[0].split()
  expanded_query=expand_query(query_preproc[1].split(),dn)
  query_terms_stemmed=query_preproc[1].split()
  query_terms_bigrams=createBigramsList(query_terms_no_stop_words)
  query_terms_bigrams_stem=createBigramsList(query_terms_stemmed)

  scores=[0]*5
  weights=[1,0.75,0.65,1]
  #----------------------------------- -- FEATURES EXTRACTION -- -----------------------------------
  sentences_score_dict={}
  for sentID in range(0,len(lines)):
    sentence_words=lines[sentID].split()
    sentence_stemmed_words=lines_stemmed[sentID].split()

    #EXTRACT QUERY RELATED FEATURES

    # ---- 1 ---- QUERY SENTENCE UNIGRAM OVERLAP
    query_sentence_unigram_overlap=0
    for wordID in range(0,len(query_preproc[0])):#len(query_terms_stemmed)):
      if query_preproc[wordID] in sentence_stemmed_words:
        query_sentence_unigram_overlap+=1
    scores[0]=query_sentence_unigram_overlap
    
    # ---- 2 ---- QUERY SENTENCE BIGRAM OVERLAP
    query_sentence_bigram_overlap=0
    sentence_bigrams_stemmed=createBigramsList(sentence_stemmed_words)
    sentence_bigrams=createBigramsList(sentence_words)
    for wordID in range(0,len(query_terms_bigrams_stem)):
      if query_terms_bigrams_stem[wordID] in sentence_bigrams_stemmed:
        query_sentence_bigram_overlap+=1
      #if query_terms_bigrams[wordID] in sentence_bigrams:
      #  query_sentence_bigram_overlap+=1
    scores[1]=query_sentence_bigram_overlap
    
    # ---- 4 ---- POS tagging with query comparision
    pos_tags=arabic_postagger.tag(sentence_words)
    proper_nouns_count=0
    proper_nouns_sentence_query_overlap=0
    for pos_tag in pos_tags:
      #if (pos_tag[1]=='NNP' or pos_tag[1]=='VBD') and pos_tag[0] in query_terms_no_stop_words:
      if pos_tag[1]=='NNP' and pos_tag[0] in query_terms_no_stop_words:
        proper_nouns_sentence_query_overlap+=1.25
    scores[2]=proper_nouns_sentence_query_overlap/len(sentence_words)
    
    # ---- 5 ---- Syntaxic simlarity between sentence and query
    syntaxic_sim=getSyntaxicSim(lines_stemmed[sentID],query_preproc[1])#query_preproc[1])
    scores[3]=syntaxic_sim
    
    final_score=0
    scores_list = [scores[i] * weights[i] for i in range(len(scores))]
    for score in scores_list:
      final_score+=score
    sentences_score_dict.update({lines_stemmed[sentID]:[final_score,sentID]})

  sentences_score_dict_sorted = sorted(sentences_score_dict.items(), key=lambda x:x[1], reverse=True)
  top_ranked_summary_sentences={}
  if N<20:
    bound = math.ceil(0.35*N) #0.35
    
  else:
    bound = math.ceil(0.25*N) #0.25
    
  for i in range(0,bound):
    top_ranked_summary_sentences.update({sentences_score_dict_sorted[i][0]:sentences_score_dict_sorted[i][1][1]})
  top_ranked_summary_sentences_sorted = sorted(top_ranked_summary_sentences.items(), key=lambda x:x[1])
  #print(sentences_score_dict_sorted)
  final_summary=""
  for sentence in top_ranked_summary_sentences_sorted:
    final_summary=final_summary+" "+map.get(sentence[0])
  print()
  print('------------------------------------------------------------- Summary -------------------------------------------------------------')
  print()
  print(final_summary)
  print()
  return final_summary


#` Evaluations`

In [ ]:
from rouge import Rouge 
def evaluate():
  res=[]
  s=1
  for i in range(s,34):
    #with open('/content/benchmark/'+str(i)+'_Article.txt', "r", encoding='utf-8') as ff:
    #  text=ff.read() 
    with open('/content/Queries/'+str(i)+'_Query.txt',"r", encoding='utf-8') as ff:
      query=ff.read() 
    print("Information Need: "+query+"\nFrom file: "+str(i)+'_Query.txt')
    #summary=sentence_sentence_similarity(text,query)
    summary=generate_summary(query,tfidf_index,docID_to_name)
    summary=summary.replace("\n","")
    summary=summary.replace("\ufeff","")
    with open('/content/Snippets/'+str(i)+'_Snippet.txt',"r", encoding='utf-8') as ff:
      snippet=ff.read() 
    snippet=snippet.replace("\n","")
    rouge = Rouge()
    scores = rouge.get_scores(summary, snippet)
    res.append(scores)
  print(res)
  r1={'r':0,'p':0,'f':0}
  r2={'r':0,'p':0,'f':0}
  rl={'r':0,'p':0,'f':0}
  res2={}

  for r in res:

    rouge1=r[0].get('rouge-1')
    r1['f']+=rouge1.get('f')
    r1['p']+=rouge1.get('p')
    r1['r']+=rouge1.get('r')

    rouge2=r[0].get('rouge-2')
    r2['f']+=rouge2.get('f')
    r2['p']+=rouge2.get('p')
    r2['r']+=rouge2.get('r')

    rougel=r[0].get('rouge-l')
    rl['f']+=rougel.get('f')
    rl['p']+=rougel.get('p')
    rl['r']+=rougel.get('r')
  
  r1['f']=r1['f']/len(res)
  r1['p']=r1['p']/len(res)
  r1['r']=r1['r']/len(res)
  r2['f']=r2['f']/len(res)
  r2['p']=r2['p']/len(res)
  r2['r']=r2['r']/len(res)
  rl['f']=rl['f']/len(res)
  rl['p']=rl['p']/len(res)
  rl['r']=rl['r']/len(res)
  
  res2.update({'rouge-1':r1})
  res2.update({'rouge-2':r2})
  res2.update({'rouge-l':rl})
  print()
  print(res2)
  return res2


In [ ]:
sum1=generate_summary("خلاف محمد صلاح وماني",tfidf_index,docID_to_name)

Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 
وأضافت الصحيفة، أن حوار دار بين محمد صلاح وساديو مانى قبل قيام اللاعب السنغالى بتسديد ركلة الجزاء، قبل أن ينجح جيمس ميلنر فى إنهاء الجدال بين ثنائى ليفربول بسحب محمد صلاح من قميصه وإبعاده عن ساديو مانى خشية تفاقم الخلاف بين الطرفين 
وتحدثت الصحيفة أن حالة من الاستياء الشديد سيطرت على محمد صلاح جراء قيام ساديو مانى بتسديد ركلة الجزاء، حيث قام بتلويح يده، مشيرة إلى أن اللاعب السنغالى أراد مصالحة "الفرعون" خلال احتفاله بالهدف قائلاً، "مو هذه لك"



In [ ]:
sum2=generate_summary("دور العيادات والممرضين فى المدرسة",tfidf_index,docID_to_name)

Currently reading document: 4_Article.txt for query: دور العيادات والممرضين فى المدرسة

------------------------------------------------------------- Summary -------------------------------------------------------------

 
توضح الدكتورة خلود السويدي، مدير إدارة الصحة المدرسية في منطقة دبي الطبية، أن الإدارة تقدم خدماتها في العيادة المركزية والعيادات الصحية، ومنها كشف شامل للتلاميذ، ومتابعة الأمراض المزمنة، والتطعيم حسب الفئة العمرية، والتأكد من نظافة البيئة المدرسية، ومن المأكولات في المقاصف المدرسية، وإعداد برامج لمكافحة السمنة ومكافحة الربو والتثقيف الصحي، والإسعافات الأولية والفحوص الرياضية، وغيرها من الخدمات التي تقدم على مدار السنة الدراسية  
ولتحقيق أقصى أنواع الرعاية تقول: توجد في العيادة المدرسية ممرضة بدوام كامل للقيام بالإسعافات المطلوبة، وعند اللزوم يتم تحويل الطالب إلى مراكز الرعاية الصحية أو مركز الصحة المدرسية، حسب قرب المسافة بين المدرسة والمركز، أما في حالة الطوارئ يتم استدعاء الإسعاف  
ويقول: يختصر دور العيادات المدرسية في الإشراف على صحة الطلاب من جميع الزوايا، وقد تم

In [ ]:
sum3=generate_summary("شروط مسابقة عام الكيخوتي للأطفال",tfidf_index,docID_to_name)

Currently reading document: 6_Article.txt for query: شروط مسابقة عام الكيخوتي للأطفال

------------------------------------------------------------- Summary -------------------------------------------------------------

  عمان- الرأي- دعا معهد ثربانتس بعمان للمشاركة بمسابقة الرسم الاولى للاطفال
المقيمين في الاردن والذين تتراوح اعمارهم ما بين 4 و12 سنة، وعنوان مسابقة الرسم
هو «عام الكيخوتي» 
وتضمنت شروط المسابقة، تنظيم مسابقة رسم للاطفال حول موضوع «عام الكيخوتي» حيث
بامكان جميع الاطفال الذين تتراوح اعمارهم ما بين 4 و12 سنة من المقيمين في عمان
او في مدن المملكة الاخرى المشاركة فيها  كل طفل مشارك في المسابقة يستطيع تقديم
رسمة او رسمتين كحد اقصى، وله مطلق الحرية في اختيار اداة الرسم سواء كانت اقلاما
خشبية او الوانا شمعية او الوان مائية او كولاجا او معاجين لطيفة، الخ es] طيلة فترة العام الدراسي الحالي 
- سوف تُقدم الجوائز يوم الخميس 26 ايار 2005 بعد انتهاء عرض فيلم الاطفال،
باحتفال قصير يسبق تسليم الجوائز الفائزة بالمسابقة



### Without QE

In [ ]:
x2=evaluate() 

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 
وأضافت الصحيفة، أن حوار دار بين محمد صلاح وساديو مانى قبل قيام اللاعب السنغالى بتسديد ركلة الجزاء، قبل أن ينجح جيمس ميلنر فى إنهاء الجدال بين ثنائى ليفربول بسحب محمد صلاح من قميصه وإبعاده عن ساديو مانى خشية تفاقم الخلاف بين الطرفين 
وتحدثت الصحيفة أن حالة من الاستياء الشديد سيطرت على محمد صلاح جراء قيام ساديو مانى بتسديد ركلة الجزاء، حيث قام بتلويح يده، مشيرة إلى أن اللاعب السنغالى أراد مصالحة "الفرعون" خلال احتفاله بالهدف قائلاً، "مو هذه لك"

Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية

------------------------------------------------------------- Summary -------------------------------------------------------------



### With QE

In [ ]:
x2=evaluate() 

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 
وأضافت الصحيفة، أن حوار دار بين محمد صلاح وساديو مانى قبل قيام اللاعب السنغالى بتسديد ركلة الجزاء، قبل أن ينجح جيمس ميلنر فى إنهاء الجدال بين ثنائى ليفربول بسحب محمد صلاح من قميصه وإبعاده عن ساديو مانى خشية تفاقم الخلاف بين الطرفين 
وتحدثت الصحيفة أن حالة من الاستياء الشديد سيطرت على محمد صلاح جراء قيام ساديو مانى بتسديد ركلة الجزاء، حيث قام بتلويح يده، مشيرة إلى أن اللاعب السنغالى أراد مصالحة "الفرعون" خلال احتفاله بالهدف قائلاً، "مو هذه لك"

Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية

------------------------------------------------------------- Summary -------------------------------------------------------------



### Previous Trials for tuning 

In [ ]:
x2=evaluate() #partial QE

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 
وأضافت الصحيفة، أن حوار دار بين محمد صلاح وساديو مانى قبل قيام اللاعب السنغالى بتسديد ركلة الجزاء، قبل أن ينجح جيمس ميلنر فى إنهاء الجدال بين ثنائى ليفربول بسحب محمد صلاح من قميصه وإبعاده عن ساديو مانى خشية تفاقم الخلاف بين الطرفين 
وتحدثت الصحيفة أن حالة من الاستياء الشديد سيطرت على محمد صلاح جراء قيام ساديو مانى بتسديد ركلة الجزاء، حيث قام بتلويح يده، مشيرة إلى أن اللاعب السنغالى أراد مصالحة "الفرعون" خلال احتفاله بالهدف قائلاً، "مو هذه لك"

Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية

------------------------------------------------------------- Summary -------------------------------------------------------------



In [ ]:
x2=evaluate() #no QE

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 
وأضافت الصحيفة، أن حوار دار بين محمد صلاح وساديو مانى قبل قيام اللاعب السنغالى بتسديد ركلة الجزاء، قبل أن ينجح جيمس ميلنر فى إنهاء الجدال بين ثنائى ليفربول بسحب محمد صلاح من قميصه وإبعاده عن ساديو مانى خشية تفاقم الخلاف بين الطرفين 
وتحدثت الصحيفة أن حالة من الاستياء الشديد سيطرت على محمد صلاح جراء قيام ساديو مانى بتسديد ركلة الجزاء، حيث قام بتلويح يده، مشيرة إلى أن اللاعب السنغالى أراد مصالحة "الفرعون" خلال احتفاله بالهدف قائلاً، "مو هذه لك"

Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية

------------------------------------------------------------- Summary -------------------------------------------------------------



In [ ]:
x2=evaluate() #0  .2 0.25

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 
وأضافت الصحيفة، أن حوار دار بين محمد صلاح وساديو مانى قبل قيام اللاعب السنغالى بتسديد ركلة الجزاء، قبل أن ينجح جيمس ميلنر فى إنهاء الجدال بين ثنائى ليفربول بسحب محمد صلاح من قميصه وإبعاده عن ساديو مانى خشية تفاقم الخلاف بين الطرفين 
وتحدثت الصحيفة أن حالة من الاستياء الشديد سيطرت على محمد صلاح جراء قيام ساديو مانى بتسديد ركلة الجزاء، حيث قام بتلويح يده، مشيرة إلى أن اللاعب السنغالى أراد مصالحة "الفرعون" خلال احتفاله بالهدف قائلاً، "مو هذه لك"

Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية

------------------------------------------------------------- Summary -------------------------------------------------------------



In [ ]:
x2=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 
وأضافت الصحيفة، أن حوار دار بين محمد صلاح وساديو مانى قبل قيام اللاعب السنغالى بتسديد ركلة الجزاء، قبل أن ينجح جيمس ميلنر فى إنهاء الجدال بين ثنائى ليفربول بسحب محمد صلاح من قميصه وإبعاده عن ساديو مانى خشية تفاقم الخلاف بين الطرفين 
وتحدثت الصحيفة أن حالة من الاستياء الشديد سيطرت على محمد صلاح جراء قيام ساديو مانى بتسديد ركلة الجزاء، حيث قام بتلويح يده، مشيرة إلى أن اللاعب السنغالى أراد مصالحة "الفرعون" خلال احتفاله بالهدف قائلاً، "مو هذه لك"

Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية

------------------------------------------------------------- Summary -------------------------------------------------------------



With QE

In [ ]:
x2=evaluate() #adjusted weights only difference is LCS instead of SimilarityOrder

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 
وأضافت الصحيفة، أن حوار دار بين محمد صلاح وساديو مانى قبل قيام اللاعب السنغالى بتسديد ركلة الجزاء، قبل أن ينجح جيمس ميلنر فى إنهاء الجدال بين ثنائى ليفربول بسحب محمد صلاح من قميصه وإبعاده عن ساديو مانى خشية تفاقم الخلاف بين الطرفين 
وتحدثت الصحيفة أن حالة من الاستياء الشديد سيطرت على محمد صلاح جراء قيام ساديو مانى بتسديد ركلة الجزاء، حيث قام بتلويح يده، مشيرة إلى أن اللاعب السنغالى أراد مصالحة "الفرعون" خلال احتفاله بالهدف قائلاً، "مو هذه لك"

Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية

------------------------------------------------------------- Summary -------------------------------------------------------------



In [ ]:
x2=evaluate() #adjusted weights 

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 
وأضافت الصحيفة، أن حوار دار بين محمد صلاح وساديو مانى قبل قيام اللاعب السنغالى بتسديد ركلة الجزاء، قبل أن ينجح جيمس ميلنر فى إنهاء الجدال بين ثنائى ليفربول بسحب محمد صلاح من قميصه وإبعاده عن ساديو مانى خشية تفاقم الخلاف بين الطرفين 
وتحدثت الصحيفة أن حالة من الاستياء الشديد سيطرت على محمد صلاح جراء قيام ساديو مانى بتسديد ركلة الجزاء، حيث قام بتلويح يده، مشيرة إلى أن اللاعب السنغالى أراد مصالحة "الفرعون" خلال احتفاله بالهدف قائلاً، "مو هذه لك"

Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية

------------------------------------------------------------- Summary -------------------------------------------------------------



In [ ]:
x2=evaluate() #adjusted weights

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 
وأضافت الصحيفة، أن حوار دار بين محمد صلاح وساديو مانى قبل قيام اللاعب السنغالى بتسديد ركلة الجزاء، قبل أن ينجح جيمس ميلنر فى إنهاء الجدال بين ثنائى ليفربول بسحب محمد صلاح من قميصه وإبعاده عن ساديو مانى خشية تفاقم الخلاف بين الطرفين 
وتحدثت الصحيفة أن حالة من الاستياء الشديد سيطرت على محمد صلاح جراء قيام ساديو مانى بتسديد ركلة الجزاء، حيث قام بتلويح يده، مشيرة إلى أن اللاعب السنغالى أراد مصالحة "الفرعون" خلال احتفاله بالهدف قائلاً، "مو هذه لك"

Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية

------------------------------------------------------------- Summary -------------------------------------------------------------



In [ ]:
x2=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 
وتغلب ليفربول على مانشستر يونايتد بنتيجة 4 – 1، ضمن منافسات بطولة كأس الابطال الدولية، والتى شهدت مشاركة الثنائى محمد صلاح وساديو مانى فى الشوط الاول من عمر المباراة 
وتحدثت الصحيفة أن حالة من الاستياء الشديد سيطرت على محمد صلاح جراء قيام ساديو مانى بتسديد ركلة الجزاء، حيث قام بتلويح يده، مشيرة إلى أن اللاعب السنغالى أراد مصالحة "الفرعون" خلال احتفاله بالهدف قائلاً، "مو هذه لك"

Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية

------------------------------------------------------------- Summary -------------------------------------------------------------

 
بداية ظهور اللهجة العامية إنّ بداية ظهور اللهجات لم يُحدد بتاريخ

Without QE

In [ ]:
x2=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 كشفت صحيفة "مترو" الإنجليزية، أن خلافاً نشب بين النجم المصرى محمد صلاح والسنغالى ساديو مانى، خلال المباراة التى جمعت بين ناديا ليفربول ومانشستر يونايتد التى أقيمت مساء أمس بالولايات المتحدة الامريكية، على تسديد ركلة الجزاء التى تمكن من خلالها ساديو مانى من تسجيل الهدف الاول فى شباك مانشستر يونايتد 
وتحدثت الصحيفة أن حالة من الاستياء الشديد سيطرت على محمد صلاح جراء قيام ساديو مانى بتسديد ركلة الجزاء، حيث قام بتلويح يده، مشيرة إلى أن اللاعب السنغالى أراد مصالحة "الفرعون" خلال احتفاله بالهدف قائلاً، "مو هذه لك"

Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية

------------------------------------------------------------- Summa

With Query Expansion

In [ ]:
x=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 
ذكرت الصحيفة، أن محمد صلاح الذى تسبب فى احتساب ركلة الجزاء، كان يطمح لتسديدها قبل أن يفاجئ بحصول ساديو مانى على الكرة متجهاً صوب نقطة الجزاء تمهيداً لتسديدها، وهو الامر الذى أثار غضب اللاعب المصري 
وأضافت الصحيفة، أن حوار دار بين محمد صلاح وساديو مانى قبل قيام اللاعب السنغالى بتسديد ركلة الجزاء، قبل أن ينجح جيمس ميلنر فى إنهاء الجدال بين ثنائى ليفربول بسحب محمد صلاح من قميصه وإبعاده عن ساديو مانى خشية تفاقم الخلاف بين الطرفين

Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية

------------------------------------------------------------- Summary -------------------------------------------------------------

 
بداية ظهور الله

In [ ]:
x=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 
وأضافت الصحيفة، أن حوار دار بين محمد صلاح وساديو مانى قبل قيام اللاعب السنغالى بتسديد ركلة الجزاء، قبل أن ينجح جيمس ميلنر فى إنهاء الجدال بين ثنائى ليفربول بسحب محمد صلاح من قميصه وإبعاده عن ساديو مانى خشية تفاقم الخلاف بين الطرفين 
وتحدثت الصحيفة أن حالة من الاستياء الشديد سيطرت على محمد صلاح جراء قيام ساديو مانى بتسديد ركلة الجزاء، حيث قام بتلويح يده، مشيرة إلى أن اللاعب السنغالى أراد مصالحة "الفرعون" خلال احتفاله بالهدف قائلاً، "مو هذه لك"

Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية

------------------------------------------------------------- Summary -------------------------------------------------------------



All measures 1-small portions 2-big portions

In [ ]:
x=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 
وأضافت الصحيفة، أن حوار دار بين محمد صلاح وساديو مانى قبل قيام اللاعب السنغالى بتسديد ركلة الجزاء، قبل أن ينجح جيمس ميلنر فى إنهاء الجدال بين ثنائى ليفربول بسحب محمد صلاح من قميصه وإبعاده عن ساديو مانى خشية تفاقم الخلاف بين الطرفين 
وتحدثت الصحيفة أن حالة من الاستياء الشديد سيطرت على محمد صلاح جراء قيام ساديو مانى بتسديد ركلة الجزاء، حيث قام بتلويح يده، مشيرة إلى أن اللاعب السنغالى أراد مصالحة "الفرعون" خلال احتفاله بالهدف قائلاً، "مو هذه لك"

Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية

------------------------------------------------------------- Summary -------------------------------------------------------------



In [ ]:
x=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 كشفت صحيفة "مترو" الإنجليزية، أن خلافاً نشب بين النجم المصرى محمد صلاح والسنغالى ساديو مانى، خلال المباراة التى جمعت بين ناديا ليفربول ومانشستر يونايتد التى أقيمت مساء أمس بالولايات المتحدة الامريكية، على تسديد ركلة الجزاء التى تمكن من خلالها ساديو مانى من تسجيل الهدف الاول فى شباك مانشستر يونايتد 
وأضافت الصحيفة، أن حوار دار بين محمد صلاح وساديو مانى قبل قيام اللاعب السنغالى بتسديد ركلة الجزاء، قبل أن ينجح جيمس ميلنر فى إنهاء الجدال بين ثنائى ليفربول بسحب محمد صلاح من قميصه وإبعاده عن ساديو مانى خشية تفاقم الخلاف بين الطرفين 
وتحدثت الصحيفة أن حالة من الاستياء الشديد سيطرت على محمد صلاح جراء قيام ساديو مانى بتسديد ركلة الجزاء، حيث قام بتلويح يده، مشيرة إلى أن اللاعب السنغالى أراد مصالحة "الفرعون" خلال احتفاله بالهدف قا

No Manh and Leve and QE

In [ ]:
x=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 كشفت صحيفة "مترو" الإنجليزية، أن خلافاً نشب بين النجم المصرى محمد صلاح والسنغالى ساديو مانى، خلال المباراة التى جمعت بين ناديا ليفربول ومانشستر يونايتد التى أقيمت مساء أمس بالولايات المتحدة الامريكية، على تسديد ركلة الجزاء التى تمكن من خلالها ساديو مانى من تسجيل الهدف الاول فى شباك مانشستر يونايتد 
وأضافت الصحيفة، أن حوار دار بين محمد صلاح وساديو مانى قبل قيام اللاعب السنغالى بتسديد ركلة الجزاء، قبل أن ينجح جيمس ميلنر فى إنهاء الجدال بين ثنائى ليفربول بسحب محمد صلاح من قميصه وإبعاده عن ساديو مانى خشية تفاقم الخلاف بين الطرفين 
وتحدثت الصحيفة أن حالة من الاستياء الشديد سيطرت على محمد صلاح جراء قيام ساديو مانى بتسديد ركلة الجزاء، حيث قام بتلويح يده، مشيرة إلى أن اللاعب السنغالى أراد مصالحة "الفرعون" خلال احتفاله بالهدف قا

In [ ]:
x=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 
وأضافت الصحيفة، أن حوار دار بين محمد صلاح وساديو مانى قبل قيام اللاعب السنغالى بتسديد ركلة الجزاء، قبل أن ينجح جيمس ميلنر فى إنهاء الجدال بين ثنائى ليفربول بسحب محمد صلاح من قميصه وإبعاده عن ساديو مانى خشية تفاقم الخلاف بين الطرفين 
وتحدثت الصحيفة أن حالة من الاستياء الشديد سيطرت على محمد صلاح جراء قيام ساديو مانى بتسديد ركلة الجزاء، حيث قام بتلويح يده، مشيرة إلى أن اللاعب السنغالى أراد مصالحة "الفرعون" خلال احتفاله بالهدف قائلاً، "مو هذه لك"

Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية

------------------------------------------------------------- Summary -------------------------------------------------------------



In [ ]:
x=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 
ذكرت الصحيفة، أن محمد صلاح الذى تسبب فى احتساب ركلة الجزاء، كان يطمح لتسديدها قبل أن يفاجئ بحصول ساديو مانى على الكرة متجهاً صوب نقطة الجزاء تمهيداً لتسديدها، وهو الامر الذى أثار غضب اللاعب المصري 
وأضافت الصحيفة، أن حوار دار بين محمد صلاح وساديو مانى قبل قيام اللاعب السنغالى بتسديد ركلة الجزاء، قبل أن ينجح جيمس ميلنر فى إنهاء الجدال بين ثنائى ليفربول بسحب محمد صلاح من قميصه وإبعاده عن ساديو مانى خشية تفاقم الخلاف بين الطرفين 
وتحدثت الصحيفة أن حالة من الاستياء الشديد سيطرت على محمد صلاح جراء قيام ساديو مانى بتسديد ركلة الجزاء، حيث قام بتلويح يده، مشيرة إلى أن اللاعب السنغالى أراد مصالحة "الفرعون" خلال احتفاله بالهدف قائلاً، "مو هذه لك"

Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently readi

In [ ]:
x=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 
ذكرت الصحيفة، أن محمد صلاح الذى تسبب فى احتساب ركلة الجزاء، كان يطمح لتسديدها قبل أن يفاجئ بحصول ساديو مانى على الكرة متجهاً صوب نقطة الجزاء تمهيداً لتسديدها، وهو الامر الذى أثار غضب اللاعب المصري 
وأضافت الصحيفة، أن حوار دار بين محمد صلاح وساديو مانى قبل قيام اللاعب السنغالى بتسديد ركلة الجزاء، قبل أن ينجح جيمس ميلنر فى إنهاء الجدال بين ثنائى ليفربول بسحب محمد صلاح من قميصه وإبعاده عن ساديو مانى خشية تفاقم الخلاف بين الطرفين

Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية

------------------------------------------------------------- Summary -------------------------------------------------------------

 
بداية ظهور الله

In [ ]:
x=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 
وأضافت الصحيفة، أن حوار دار بين محمد صلاح وساديو مانى قبل قيام اللاعب السنغالى بتسديد ركلة الجزاء، قبل أن ينجح جيمس ميلنر فى إنهاء الجدال بين ثنائى ليفربول بسحب محمد صلاح من قميصه وإبعاده عن ساديو مانى خشية تفاقم الخلاف بين الطرفين 
وتحدثت الصحيفة أن حالة من الاستياء الشديد سيطرت على محمد صلاح جراء قيام ساديو مانى بتسديد ركلة الجزاء، حيث قام بتلويح يده، مشيرة إلى أن اللاعب السنغالى أراد مصالحة "الفرعون" خلال احتفاله بالهدف قائلاً، "مو هذه لك"

Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية

------------------------------------------------------------- Summary -------------------------------------------------------------



In [ ]:
x=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني

------------------------------------------------------------- Summary -------------------------------------------------------------

 
وأضافت الصحيفة، أن حوار دار بين محمد صلاح وساديو مانى قبل قيام اللاعب السنغالى بتسديد ركلة الجزاء، قبل أن ينجح جيمس ميلنر فى إنهاء الجدال بين ثنائى ليفربول بسحب محمد صلاح من قميصه وإبعاده عن ساديو مانى خشية تفاقم الخلاف بين الطرفين 
وتحدثت الصحيفة أن حالة من الاستياء الشديد سيطرت على محمد صلاح جراء قيام ساديو مانى بتسديد ركلة الجزاء، حيث قام بتلويح يده، مشيرة إلى أن اللاعب السنغالى أراد مصالحة "الفرعون" خلال احتفاله بالهدف قائلاً، "مو هذه لك"

Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية

------------------------------------------------------------- Summary -------------------------------------------------------------



In [ ]:
x=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني
Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية
Information Need:  اكتشاف مقبرة الفرعون الذهبي توت عنخ آمون
From file: 3_Query.txt
Currently reading document: 3_Article.txt for query:  اكتشاف مقبرة الفرعون الذهبي توت عنخ آمون
Information Need: دور العيادات والممرضين فى المدرسة
From file: 4_Query.txt
Currently reading document: 4_Article.txt for query: دور العيادات والممرضين فى المدرسة
Information Need: عوامل والسبب وراء حدوث الهذيان
From file: 5_Query.txt
Currently reading document: 5_Article.txt for query: عوامل والسبب وراء حدوث الهذيان
Information Need:  شروط مسابقة عام الكيخوتي للأطفال

From file: 6_Query.txt
Currently reading document: 6_Article.txt for query:  شروط مسابقة عام الكيخوتي للأطفال

Information Need: انواع الهذيان وتصنيفاته الطبية
From file

In [ ]:
x=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني
Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية
Information Need:  اكتشاف مقبرة الفرعون الذهبي توت عنخ آمون
From file: 3_Query.txt
Currently reading document: 3_Article.txt for query:  اكتشاف مقبرة الفرعون الذهبي توت عنخ آمون
Information Need: دور العيادات والممرضين فى المدرسة
From file: 4_Query.txt
Currently reading document: 4_Article.txt for query: دور العيادات والممرضين فى المدرسة
Information Need: عوامل والسبب وراء حدوث الهذيان
From file: 5_Query.txt
Currently reading document: 5_Article.txt for query: عوامل والسبب وراء حدوث الهذيان
Information Need:  شروط مسابقة عام الكيخوتي للأطفال

From file: 6_Query.txt
Currently reading document: 6_Article.txt for query:  شروط مسابقة عام الكيخوتي للأطفال

Information Need: انواع الهذيان وتصنيفاته الطبية
From file

In [ ]:
x=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني
Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية
Information Need:  اكتشاف مقبرة الفرعون الذهبي توت عنخ آمون
From file: 3_Query.txt
Currently reading document: 3_Article.txt for query:  اكتشاف مقبرة الفرعون الذهبي توت عنخ آمون
Information Need: دور العيادات والممرضين فى المدرسة
From file: 4_Query.txt
Currently reading document: 4_Article.txt for query: دور العيادات والممرضين فى المدرسة
Information Need: عوامل والسبب وراء حدوث الهذيان
From file: 5_Query.txt
Currently reading document: 5_Article.txt for query: عوامل والسبب وراء حدوث الهذيان
Information Need:  شروط مسابقة عام الكيخوتي للأطفال

From file: 6_Query.txt
Currently reading document: 6_Article.txt for query:  شروط مسابقة عام الكيخوتي للأطفال

Information Need: انواع الهذيان وتصنيفاته الطبية
From file

In [ ]:
x=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني
Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية
Information Need:  اكتشاف مقبرة الفرعون الذهبي توت عنخ آمون
From file: 3_Query.txt
Currently reading document: 3_Article.txt for query:  اكتشاف مقبرة الفرعون الذهبي توت عنخ آمون
Information Need: دور العيادات والممرضين فى المدرسة
From file: 4_Query.txt
Currently reading document: 4_Article.txt for query: دور العيادات والممرضين فى المدرسة
Information Need: عوامل والسبب وراء حدوث الهذيان
From file: 5_Query.txt
Currently reading document: 5_Article.txt for query: عوامل والسبب وراء حدوث الهذيان
Information Need:  شروط مسابقة عام الكيخوتي للأطفال

From file: 6_Query.txt
Currently reading document: 6_Article.txt for query:  شروط مسابقة عام الكيخوتي للأطفال

Information Need: انواع الهذيان وتصنيفاته الطبية
From file

In [ ]:
x=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني
Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية
Information Need:  اكتشاف مقبرة الفرعون الذهبي توت عنخ آمون
From file: 3_Query.txt
Currently reading document: 3_Article.txt for query:  اكتشاف مقبرة الفرعون الذهبي توت عنخ آمون
Information Need: دور العيادات والممرضين فى المدرسة
From file: 4_Query.txt
Currently reading document: 4_Article.txt for query: دور العيادات والممرضين فى المدرسة
Information Need: عوامل والسبب وراء حدوث الهذيان
From file: 5_Query.txt
Currently reading document: 5_Article.txt for query: عوامل والسبب وراء حدوث الهذيان
Information Need:  شروط مسابقة عام الكيخوتي للأطفال

From file: 6_Query.txt
Currently reading document: 6_Article.txt for query:  شروط مسابقة عام الكيخوتي للأطفال

Information Need: انواع الهذيان وتصنيفاته الطبية
From file

In [ ]:
x=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني
Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية
Information Need:  اكتشاف مقبرة الفرعون الذهبي توت عنخ آمون
From file: 3_Query.txt
Currently reading document: 3_Article.txt for query:  اكتشاف مقبرة الفرعون الذهبي توت عنخ آمون
Information Need: دور العيادات والممرضين فى المدرسة
From file: 4_Query.txt
Currently reading document: 4_Article.txt for query: دور العيادات والممرضين فى المدرسة
Information Need: عوامل والسبب وراء حدوث الهذيان
From file: 5_Query.txt
Currently reading document: 5_Article.txt for query: عوامل والسبب وراء حدوث الهذيان
Information Need:  شروط مسابقة عام الكيخوتي للأطفال

From file: 6_Query.txt
Currently reading document: 6_Article.txt for query:  شروط مسابقة عام الكيخوتي للأطفال

Information Need: انواع الهذيان وتصنيفاته الطبية
From file

In [ ]:
x=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني
Information Need: بداية ظهور اللهجة العامية
From file: 2_Query.txt
Currently reading document: 2_Article.txt for query: بداية ظهور اللهجة العامية
Information Need:  اكتشاف مقبرة الفرعون الذهبي توت عنخ آمون
From file: 3_Query.txt
Currently reading document: 3_Article.txt for query:  اكتشاف مقبرة الفرعون الذهبي توت عنخ آمون
Information Need: دور العيادات والممرضين فى المدرسة
From file: 4_Query.txt
Currently reading document: 4_Article.txt for query: دور العيادات والممرضين فى المدرسة
Information Need: عوامل والسبب وراء حدوث الهذيان
From file: 5_Query.txt
Currently reading document: 5_Article.txt for query: عوامل والسبب وراء حدوث الهذيان
Information Need:  شروط مسابقة عام الكيخوتي للأطفال

From file: 6_Query.txt
Currently reading document: 6_Article.txt for query:  شروط مسابقة عام الكيخوتي للأطفال

Information Need: انواع الهذيان وتصنيفاته الطبية
From file

In [ ]:
x=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني
[('وضف  صحف  ن  حور  دور  حمد  صلح  سدو  مني  قوم  عبب  سنغل  سدد  ركل  جزز  ن  نجح  جمس  لنر  فيي  نهء  جدل  ثنء  فربل  سحب  حمد  صلح  قمص  بعد  سدو  مني  خشي  فقم  خلف  طرف ', [4.356680380954139, 3]), ('كشف  صحف  ترر  جلز  ن  نشب  نجم  صري  حمد  صلح  سنغل  سدو  مني  خلال  برو  ولت  جمع  ندو  فربل  منشستر  نتد  ولت  وقم  ومس  بلل  حدد  مرك  سدد  ركل  جزز  ولت  مكن  خلال  سدو  مني  سجل  هدف  وول  فيي  شبك  نشستر  نتد ', [3.473182602776264, 0]), ('غلب  فربل  نشستر  نتد  نوج  4  1  ضمن  نفس  طول  كوس  بطل  دلو  ولت  شهد  شرك  ثنء  حمد  صلح  سدو  مني  فيي  شوط  وول  عمر  برو ', [3.417603403931456, 1]), ('حدث  صحف  ن  حول  لست  شدد  سطر  حمد  صلح  جرر  قوم  سدو  مني  سدد  ركل  جزز  لوح  يده  شير  ولي  ن  عبب  سنغل  رود  صلح  فرع  خلال  حتفل  هدف  قءل  م ', [3.3720834393722257, 4]), ('ذكر  صحف  ن  حمد  صلح  لذي  سبب  فيي  حتسب  ركل  جزز  طمح  سدد  ن  فجء  ح

In [ ]:
x=evaluate()

Information Need: خلاف محمد صلاح وماني
From file: 1_Query.txt
Currently reading document: 1_Article.txt for query: خلاف محمد صلاح وماني
[('وضف  صحف  ن  حور  دور  حمد  صلح  وسد  مني  قوم  عبب  سنغل  سدد  ركل  جزز  ن  نجح  جمس  لنر  فيي  نهء  جدل  ثنء  فربل  سحب  حمد  صلح  قمص  بعد  سدو  مني  خشي  فقم  خلف  طرف ', [7.172879363954507, 3]), ('شفف  صحف  ترر  جلز  ن  شبب  نجم  صرر  حمد  صلح  سنغل  سدو  مني  خلال  برو  ولت  جمع  ندو  فربل  منشستر  نتد  ولت  وقم  مسس  بلل  حدد  مرك  سدد  ركل  جزز  ولت  مكن  خلال  سدو  مني  سجل  هدف  وول  فيي  شبب  نشستر  نتد ', [5.038442931371552, 0]), ('حدث  صحف  ن  حول  لست  شدد  طرر  حمد  صلح  جرء  قوم  سدو  مني  سدد  ركل  جزز  لوح  يده  شره  ولي  ن  عبب  سنغل  رود  صلح  فرع  خلال  حتفل  هدف  قءل  م ', [4.998397753661314, 4]), ('ذكر  صحف  ن  حمد  صلح  لذي  سبب  فيي  حتسب  ركل  جزز  طمح  سدد  ن  فجء  حصل  سدو  مني  كرر  جهه  صوب  نقط  جزز  مهد  سدد  مرر  لذي  ثور  غضب  عبب  مصر ', [4.965807001967098, 2]), ('غلب  فربل  نشستر  نتد  نوج  4  1  ضمن  نفس  بطل  كو

In [ ]:
{'rouge-1': {'r': 0.7777126108796603, 'p': 0.4787918246099692, 'f': 0.5543290572221741}, 'rouge-2': {'r': 0.742803580182024, 'p': 0.43180419532692865, 'f': 0.5038373646285096}, 'rouge-l': {'r': 0.7715027368533997, 'p': 0.4740596942238352, 'f': 0.5491693475273647}}
{'rouge-1': {'r': 0.7777126108796603, 'p': 0.482723147524343, 'f': 0.5585364590238908}, 'rouge-2': {'r': 0.742803580182024, 'p': 0.43603199367237405, 'f': 0.5083899556431993}, 'rouge-l': {'r': 0.7720854874361501, 'p': 0.4781641773113691, 'f': 0.5536437363802095}}
{'rouge-1': {'r': 0.7797707546816413, 'p': 0.475674969259999, 'f': 0.5548122097383649}, 'rouge-2': {'r': 0.7466984224450448, 'p': 0.42902002322590976, 'f': 0.5054035777653123}, 'rouge-l': {'r': 0.77396833560829, 'p': 0.4709886860605909, 'f': 0.549901541079463}}
{'rouge-1': {'r': 0.7778709972004885, 'p': 0.47655014342170965, 'f': 0.5543915404017584}, 'rouge-2': {'r': 0.741036325716812, 'p': 0.4280183792342272, 'f': 0.5033320995438112}, 'rouge-l': {'r': 0.769181271914191, 'p': 0.47040001218180716, 'f': 0.5474975976049379}}
{'rouge-1': {'r': 0.7929416089656077, 'p': 0.43893452346073736, 'f': 0.5327261500994156}, 'rouge-2': {'r': 0.758028250592192, 'p': 0.39222323350627153, 'f': 0.4830180971621872}, 'rouge-l': {'r': 0.7862136657491912, 'p': 0.4345221408965383, 'f': 0.5277438593922775}}
{'rouge-1': {'r': 0.811364951225389, 'p': 0.4349762594736286, 'f': 0.5311055954126118}, 'rouge-2': {'r': 0.7791720544051685, 'p': 0.3907286776048752, 'f': 0.4836240706790773}, 'rouge-l': {'r': 0.8053997139770492, 'p': 0.4307720998201222, 'f': 0.5263379892726187}}



#`Other Trials`

In [ ]:
#query='معانة بيتهوفن واستخدامه للموسيقى للتغلب على المشاكل'
#query='اعراض الإصابة بالهذيان'
#query='أعمق نهر فى العالم'
#query='بداية ظهور اللهجة العامية' 
#query='اكتشاف مقبرة توت عنخ أمون'
#query='خلاف محمد صلاح وماني'
#query='أمان الاطفال فى الأماكن العامة'

#doc=text[0]
#for t in range(1,len(text)):
#  doc=doc+' '+text[t]

In [ ]:
getOccurences(vec1)

([], 3)

In [ ]:
sent= "شُرْبُ القِطِّ الحَليبُ اللَّذيذُ"

In [ ]:
from nltk.tag.stanford import StanfordPOSTagger as POS_Tag
arabic_postagger = POS_Tag('arabic.tagger', 'stanford-postagger.jar')
sentence = '.شرب القط الحليب اللذيذ'
query='أعمق نهر فى العالم'

print(arabic_postagger.tag(query.split()))

/usr/local/lib/python3.7/dist-packages/nltk/tag/stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)


[('', 'أعمق/VBD'), ('', 'نهر/NN'), ('', 'فى/IN'), ('', 'العالم/DTNN')]


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw')
nltk.download('omw-1.4')
from nltk.corpus import wordnet as wn
for synset in wn.synsets('عمق',lang=('arb')):
    for hypo in synset.hyponyms():
         for lemma in hypo.lemmas("arb"):
             print(str(lemma).split('.')[3].replace("')",""))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
أسْرع
تسارع
سرع
عجل
عجّل


In [ ]:
snippets=[]
avg_len=0
for i in range(1,24):
  with open('/content/benchmark/'+str(i)+'_Snippet.txt',"r", encoding='utf-8') as ff:
    snippet=ff.read() 
  snippets.append(snippet)
  avg_len+=len(snippet.split("."))
avg_len=avg_len/23
print(avg_len)

In [ ]:
j=0
new_dict={}
for k,v in domain_knowledge_dict.items():
  k=k.rstrip()
  l=[]
  for vv in v:
    l.append(vv.rstrip())
  new_dict.update({k:l})

In [ ]:

json_object = json.dumps(new_dict)
  
# Writing to sample.json
with open('/content/___domainknowledge__.txt',"w") as outfile:
    outfile.write(json_object)
outfile.close()

In [ ]:
s=1
j=0
for i in range(s,34):
  with open('/content/Queries/'+str(i)+'_Query.txt',"r", encoding='utf-8') as ff:
    query=ff.read() 
  query_preproc=query_preprocess(query)
  query_terms_no_stop_words=query_preproc[0].split()
  query_terms_stemmed=query_preproc[1].split()
  for qt in query_terms_stemmed:
    if qt in dn:
        j+=1
        print(qt)
print(j)

حمد
صلح
بدو
ظهر
لهج
عوم
قبر
فرع
توت
عنخ
دور
عود
مرض
فيي
درس
عمل
سبب
حدث
شرط
سبق
عوم
طفل
نوع
صنف
طبب
مور
رفع
فرص
عمق
نهر
فيي
طول
نهر
نيل
تهف
خدم
وسق
غلب
شكل
هدف
ورش
حور
فيي
خطط
عمن
برد
سوي
طوع
شمس
خطر
كعب
صحح
قوم
لمر
جدد
دخن
طفل
طفل
فيي
طفل
فيي
حمم
طفل
فيي
عوم
طفل
فيي
سور
مرر
رشح
خلد
حيي
جمهر
نسب
خدم
سرع
تحد
ورب
شرك
مصر
حرب
يمن
عدل
سقط
لمس
ثلج
مطر
ككب
بلغ
حجم
شمس
حدي
رود
نجم
مرض
سبب
مرض
سبب
فرس
كثر
مكن
حدد
شكل
حجم
فرس
105


In [ ]:
s=1
i=1
for i in range(1,34):
  with open('/content/Queries/'+str(i)+'_Query.txt',"r", encoding='utf-8') as ff:
    query=ff.read() 
    print(query)
  query_preproc=query_preprocess(query)
  query_terms_no_stop_words=query_preproc[0].split()
  query_terms_stemmed=query_preproc[1].split()
  expanded_query=expand_query(query_terms_stemmed,dn_bn)
  print(expanded_query[0])
  print(expanded_query[1])
  print()

خلاف محمد صلاح وماني
['خلف', 'حمد', 'صلح', 'ومن', 'ءثر', 'ركز', 'حفظ', 'ولث']
خلف حمد صلح ومن ءثر ركز حفظ ولث

بداية ظهور اللهجة العامية
['بدو', 'ظهر', 'لهج', 'عوم', 'ثمم', 'نشط', 'كتب', 'مره']
بدو ظهر لهج عوم ثمم نشط كتب مره

 اكتشاف مقبرة الفرعون الذهبي توت عنخ آمون
['كتشف', 'قبر', 'فرع', 'ذهب', 'توت', 'عنخ', 'مون', 'شهد', 'ختم', 'كتشف', 'لدى', 'خرر', 'لوت']
كتشف قبر فرع ذهب توت عنخ مون شهد ختم كتشف لدى خرر لوت

دور العيادات والممرضين فى المدرسة
['دور', 'عود', 'مرض', 'فيي', 'درس', 'ضمم', 'حلو', 'شوء', 'طلب', 'لبرفسر']
دور عود مرض فيي درس ضمم حلو شوء طلب لبرفسر

عوامل والسبب وراء حدوث الهذيان
['عمل', 'سبب', 'روء', 'حدث', 'هذذ', 'بءه', 'قدم', 'لكم', 'ءدد']
عمل سبب روء حدث هذذ بءه قدم لكم ءدد

 شروط مسابقة عام الكيخوتي للأطفال

['شرط', 'سبق', 'عوم', 'كخت', 'طفل', 'عنف', 'عدل', 'مره', 'جهه']
شرط سبق عوم كخت طفل عنف عدل مره جهه

انواع الهذيان وتصنيفاته الطبية
['نوع', 'هذذ', 'صنف', 'طبب', 'لزم', 'طله', 'وفر']
نوع هذذ صنف طبب لزم طله وفر

الأمور التي قد ترفع من فرص الإصابة بالهذيان

['مور',

In [ ]:
def expand_query(query_terms_stemmed, dn):
  added_terms=[]
  for qt in query_terms_stemmed:
    if qt in dn:
      added_terms.append(dn.get(qt)[0])
      added_terms.append(dn.get(qt)[1])
      added_terms.append(dn.get(qt)[3])
  expanded_query=query_terms_stemmed+added_terms
  return 

In [ ]:
def generate_summary2(query,tf_idf_index,docID_to_name):
  #Read Article 
  doc=selectDocument(query,tf_idf_index,docID_to_name)
  print("Currently reading document: "+doc[0]+" for query: "+query)
  i=doc[1]
  with open('/content/Articles/'+str(i)+'_Article.txt', "r", encoding='utf-8') as ff:
    text=ff.read() 
  #----------------------------------------------ARTICLE--------------------------------------------
  text_preproc=text_preprocess(text)
  lines=text_preproc[0]
  map=text_preproc[2]
  lines_stemmed=text_preproc[1]
  N=len(lines)
  doc_words_freq_dict={}
  for line in lines_stemmed:
    words=line.split()
    for word in words:
      if word not in doc_words_freq_dict:
        doc_words_freq_dict.update({word:1})
      else:
        c=doc_words_freq_dict.get(word)+1
        doc_words_freq_dict.update({word:c})
  
  #-----------------------------------------------QUERY---------------------------------------------
  query_preproc=query_preprocess(query)
  query_terms_no_stop_words=query_preproc[0].split()
  query_terms_stemmed=query_preproc[1].split()
  query_terms_bigrams=createBigramsList(query_terms_no_stop_words)
  query_terms_bigrams_stem=createBigramsList(query_terms_stemmed)

  scores=[0]*5
  #weights=[0.65,0.75,0.95,0.60,0.95]
  weights=[0.75,0.95,0,0.80,2]
  #----------------------------------- -- FEATURES EXTRACTION -- -----------------------------------
  sentences_score_dict={}
  for sentID in range(0,len(lines)):
    sentence_words=lines[sentID].split()
    sentence_stemmed_words=lines_stemmed[sentID].split()

    #EXTRACT QUERY RELATED FEATURES

    # ---- 1 ---- QUERY SENTENCE UNIGRAM OVERLAP
    query_sentence_unigram_overlap=0
    for wordID in range(0,len(query_terms_stemmed)):
      if query_terms_stemmed[wordID] in sentence_stemmed_words:
        query_sentence_unigram_overlap+=1
      #if query_terms_no_stop_words[wordID] in sentence_words:
      #  query_sentence_unigram_overlap+=1
    scores[0]=query_sentence_unigram_overlap
    
    # ---- 2 ---- QUERY SENTENCE BIGRAM OVERLAP
    query_sentence_bigram_overlap=0
    sentence_bigrams_stemmed=createBigramsList(sentence_stemmed_words)
    sentence_bigrams=createBigramsList(sentence_words)
    for wordID in range(0,len(query_terms_bigrams_stem)):
      if query_terms_bigrams_stem[wordID] in sentence_bigrams_stemmed:
        query_sentence_bigram_overlap+=1
      #if query_terms_bigrams[wordID] in sentence_bigrams:
      #  query_sentence_bigram_overlap+=1
    scores[1]=query_sentence_bigram_overlap

    # ---- 3 ---- QUERY SENTENCE TRIGRAM OVERLAP
    query_sentence_trigram_overlap=0
    sentence_trigrams_stemmed=createTrigramsList(sentence_stemmed_words)
    sentence_trigrams=createBigramsList(sentence_words)
    query_terms_trigrams=createBigramsList(query_terms_no_stop_words)
    query_terms_trigrams_stem=createBigramsList(query_terms_stemmed)
    for wordID in range(0,len(query_terms_trigrams_stem)):
      if query_terms_trigrams_stem[wordID] in sentence_trigrams_stemmed:
        query_sentence_trigram_overlap+=1
      #if query_terms_trigrams[wordID] in sentence_trigrams:
      #  query_sentence_trigram_overlap+=1
    scores[2]=query_sentence_trigram_overlap
    
    # ---- 4 ---- POS tagging with query comparision
    pos_tags=arabic_postagger.tag(sentence_words)
    proper_nouns_count=0
    proper_nouns_sentence_query_overlap=0
    for pos_tag in pos_tags:
      if pos_tag[1]=='NNP':
        proper_nouns_sentence_query_overlap+=1
    scores[3]=proper_nouns_sentence_query_overlap/len(sentence_words)
    
    # ---- 5 ---- Syntaxic simlarity between sentence and query
    syntaxic_sim=getSyntaxicSim(lines_stemmed[sentID],query_preproc[1])
    scores[4]=syntaxic_sim
    
    final_score=0
    scores_list = [scores[i] * weights[i] for i in range(len(scores))]
    for score in scores_list:
      final_score+=score
    sentences_score_dict.update({lines_stemmed[sentID]:[final_score,sentID]})

  sentences_score_dict_sorted = sorted(sentences_score_dict.items(), key=lambda x:x[1], reverse=True)
  top_ranked_summary_sentences={}
  if N<20:
    bound = math.ceil(0.3*N)
    if bound>4:
      bound =4
  else:
    bound = math.ceil(0.25*N)
    if bound>6:
      bound=6
  for i in range(0,bound):
    top_ranked_summary_sentences.update({sentences_score_dict_sorted[i][0]:sentences_score_dict_sorted[i][1][1]})
  top_ranked_summary_sentences_sorted = sorted(top_ranked_summary_sentences.items(), key=lambda x:x[1])
  print(sentences_score_dict_sorted)
  final_summary=""
  for sentence in top_ranked_summary_sentences_sorted:
    final_summary=final_summary+" "+map.get(sentence[0])
  print()
  print('------------------------------------------------------------- Summary -------------------------------------------------------------')
  print()
  print(final_summary)
  print()
  return final_summary
